In [77]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm
import seaborn as sns
from IPython.display import display
import datetime
import numpy as np
from statsmodels import robust
from sklearn import preprocessing
from sklearn.preprocessing import Imputer
from scipy import stats
from scipy.stats import zscore

In [2]:
def null_ratio(df):
        null_count = df.isnull().sum()
        null_percent = 100 * df.isnull().sum()/len(df)
        null_table = pd.concat([null_count, null_percent], axis=1)
        null_table = null_table.rename(columns = {0 : 'Null Count', 1 : 'Null Percent'})
        return null_table.sort_values('Null Percent', ascending=0)

In [3]:
def return_all_rows(x):
    pd.set_option('display.max_rows', len(x))
    return x
    pd.reset_option('display.max_rows')
    
def return_all_columns(x):
    pd.set_option('display.max_columns', len(x))
    return x.head(5)
    pd.reset_option('display.max_columns')

In [4]:
def overview(df):
    print("Number of columns:", len(df.columns))
    print("Number of rows:", len(df.index))
    df.head(5)

In [5]:
def drop_dups(df):
    # list comprehension of the cols that end with '_y'
    y_drop = [x for x in df if x.endswith('_y')]
    df.drop(y_drop, axis=1, inplace=True)

In [6]:
def floatToString(inputValue):
    result = ('%.15f' % inputValue).rstrip('0').rstrip('.')
    return '0' if result == '-0' else result

In [7]:
def mad(array):
    """ 
    Get Median Absolute Deviation and multiple by 1.486 to mimic standard deviation
        https://www.ibm.com/support/knowledgecenter/SSWLVY_1.0.0/com.ibm.spss.analyticcatalyst.help/analytic_catalyst/modified_z.html
    Median Absolute Deviation: a "Robust" version of standard deviation.
        Indices variabililty of the sample.
        https://en.wikipedia.org/wiki/Median_absolute_deviation
    """
    array = np.ma.array(array).compressed()
    median = np.nanmedian(array)
    mad = np.nanmedian(np.abs(array - median))
    return mad

In [8]:
def meanad(array):
    """ 
    Get Mean Absolute Deviation and multiple by 1.253314 to mimic standard deviation
        https://www.ibm.com/support/knowledgecenter/SSWLVY_1.0.0/com.ibm.spss.analyticcatalyst.help/analytic_catalyst/modified_z.html
    Mean Absolute Deviation: a "Robust" version of standard deviation.
        Indices variabililty of the sample.
        https://en.wikipedia.org/wiki/Mean_absolute_deviation
    """
    array = np.ma.array(array).compressed()
    median = np.nanmedian(array)
    mad = np.nanmean(np.abs(array - median))
    return mad

In [9]:
def modified_z(array):
    try:
        try:
            
            try:
                median = np.nanmedian(array)
                denominator = mad(array) * 1.486
                array = (array - median) / denominator
                return array
            
            except:
                median = np.nanmedian(array)
                denominator = meanad(array) * 1.253314
                array = (array - median) / denominator
                return array
        
        except:
            mean = np.nanmean(array)
            denominator = np.nanstd(array)
            array = (array - mean) / denominator
            return array
    
    except:
        array = array.fillna(0)

In [10]:
def fill_null(column):
    try:
        median = np.nanmedian(column)
        column = column.fillna(median)
        return column
    except:
        return column

In [11]:
def impute_null(column):
    try:
        imp = Imputer(missing_values='NaN', strategy='median', axis=0)
        imp.fit(column)
        column = imp.transform(column)
        return column
    except:
        return column

In [12]:
def clip_outliers(column):
    
    # Use try in case all null column
    try:
        floor = column.quantile(0.02)
        ceiling = column.quantile(0.98)
        column = column.clip(floor, ceiling)
        return column
    # If error, return as is
    except:
        return column

In [13]:
# df - Load
df = pd.read_csv('c:/users/sconner/Desktop/W205_Final/sample_data/Sample_Financial_Ratios_Firm_Level.csv')

In [14]:
# Link Table - Load
link_table = pd.read_csv('c:/users/sconner/Desktop/W205_Final/sample_data/Sample_CRSP_Compustat_Merged_Linking_Table.csv')

In [15]:
# Link Table - Clean - LINKENDDT
# Replace 'E' values with future date (current unique ID)
link_table['LINKENDDT'] = link_table['LINKENDDT'].str.replace('E','20200101').astype(int)

In [16]:
# Link Table - Clean - LPERMNO
# Convert PERMNO from int to object
link_table['LPERMNO'] = link_table['LPERMNO'].astype(object)

In [17]:
# Link Table - Clean - LINKDT & LINKENDDT
# Get start and end dates for link to determine identification during given time
link_table['LINKDT'] = pd.to_datetime(link_table['LINKDT'], format='%Y%m%d')
link_table['LINKENDDT'] = pd.to_datetime(link_table['LINKENDDT'], format='%Y%m%d')

In [18]:
# Merge - df & Link Table
# Merge financial ratio suite with link table to build more foreign keys
df = pd.merge(df, link_table, on='gvkey', how='left', suffixes=('', '_y'))
# Release Memory
del link_table

In [19]:
# df - Enrich - Time Features
# Convert to time
df['public_date'] = pd.to_datetime(df['public_date'])
# Create year
df['year'] = df['public_date'].dt.year
# Create month
df['month'] = df['public_date'].dt.month
# Create year-month
df["year-month"] = df['public_date'].apply(lambda x: x.strftime('%Y-%m'))

In [20]:
# df - Enrich - Time-based Unique Identifiers
# Create Unique Identifier for links from other databases for specific date-time
df["GVKEY-year-month"] = df["gvkey"].map(str) + "-" + df["year-month"]
df["CUSIP-year-month"] = df["cusip"].map(str) + "-" + df["year-month"]
df["TIC-year-month"] = df["tic"].map(str) + "-" + df["year-month"]
df["PERMNO-year-month"] = df["LPERMNO"].map(str) + "-" + df["year-month"]

In [21]:
# df - Subset - Correct Unique ID's per public data
# Filter dataframe to rows of unique identifiers during correct link range
df = df[(df.public_date >= df.LINKDT) & (df.public_date <= df.LINKENDDT)]

In [22]:
# CRSP - Load
# Load CRSP/Compustat Merged Database - Security Monthly from Wharton
CRSP_comp_merge = pd.read_csv('c:/users/sconner/desktop/W205_Final/sample_data/Sample_CRSP_Compustat_Merged_Security_Monthly.csv', low_memory=False)

In [23]:
# CRSP - Enrich - Time Features

# Convert to date-time
CRSP_comp_merge['datadate'] = pd.to_datetime(CRSP_comp_merge['datadate'])
# Create year
CRSP_comp_merge['year'] = CRSP_comp_merge['datadate'].dt.year
# Create month
CRSP_comp_merge['month'] = CRSP_comp_merge['datadate'].dt.month
# Create year-month
CRSP_comp_merge["year-month"] = CRSP_comp_merge['datadate'].apply(lambda x: x.strftime('%Y-%m'))

In [24]:
# CRSP - Enrich - Time-based Unique Identifiers
# CRSP - Unique ID in time (GVKEY + time-month)
CRSP_comp_merge["GVKEY-year-month"] = CRSP_comp_merge["GVKEY"].map(str) + "-" + CRSP_comp_merge["year-month"]

In [25]:
# CRSP - Sort - Date and GVKEY
CRSP_comp_merge = CRSP_comp_merge.sort_values(by=['GVKEY','datadate'], ascending=[True,True])

In [26]:
# CRSP - Enrich - Forward (PRCCM -- Price - Close - Monthly) 1 – 36 months
CRSP_comp_merge['forward_one_month_prccm'] = CRSP_comp_merge.groupby('GVKEY')['prccm'].shift(-1)
CRSP_comp_merge['forward_two_month_prccm'] = CRSP_comp_merge.groupby('GVKEY')['prccm'].shift(-2)
CRSP_comp_merge['forward_three_month_prccm'] = CRSP_comp_merge.groupby('GVKEY')['prccm'].shift(-3)
CRSP_comp_merge['forward_four_month_prccm'] = CRSP_comp_merge.groupby('GVKEY')['prccm'].shift(-4)
CRSP_comp_merge['forward_five_month_prccm'] = CRSP_comp_merge.groupby('GVKEY')['prccm'].shift(-5)
CRSP_comp_merge['forward_six_month_prccm'] = CRSP_comp_merge.groupby('GVKEY')['prccm'].shift(-6)
CRSP_comp_merge['forward_seven_month_prccm'] = CRSP_comp_merge.groupby('GVKEY')['prccm'].shift(-7)
CRSP_comp_merge['forward_eight_month_prccm'] = CRSP_comp_merge.groupby('GVKEY')['prccm'].shift(-8)
CRSP_comp_merge['forward_nine_month_prccm'] = CRSP_comp_merge.groupby('GVKEY')['prccm'].shift(-9)
CRSP_comp_merge['forward_ten_month_prccm'] = CRSP_comp_merge.groupby('GVKEY')['prccm'].shift(-10)
CRSP_comp_merge['forward_eleven_month_prccm'] = CRSP_comp_merge.groupby('GVKEY')['prccm'].shift(-11)
CRSP_comp_merge['forward_twelve_month_prccm'] = CRSP_comp_merge.groupby('GVKEY')['prccm'].shift(-12)
CRSP_comp_merge['forward_thirteen_month_prccm'] = CRSP_comp_merge.groupby('GVKEY')['prccm'].shift(-13)
CRSP_comp_merge['forward_fourteen_month_prccm'] = CRSP_comp_merge.groupby('GVKEY')['prccm'].shift(-14)
CRSP_comp_merge['forward_fifteen_month_prccm'] = CRSP_comp_merge.groupby('GVKEY')['prccm'].shift(-15)
CRSP_comp_merge['forward_sixteen_month_prccm'] = CRSP_comp_merge.groupby('GVKEY')['prccm'].shift(-16)
CRSP_comp_merge['forward_seventeen_month_prccm'] = CRSP_comp_merge.groupby('GVKEY')['prccm'].shift(-17)
CRSP_comp_merge['forward_eighteen_month_prccm'] = CRSP_comp_merge.groupby('GVKEY')['prccm'].shift(-18)
CRSP_comp_merge['forward_nineteen_month_prccm'] = CRSP_comp_merge.groupby('GVKEY')['prccm'].shift(-19)
CRSP_comp_merge['forward_twenty_month_prccm'] = CRSP_comp_merge.groupby('GVKEY')['prccm'].shift(-20)
CRSP_comp_merge['forward_twentyone_month_prccm'] = CRSP_comp_merge.groupby('GVKEY')['prccm'].shift(-21)
CRSP_comp_merge['forward_twentytwo_month_prccm'] = CRSP_comp_merge.groupby('GVKEY')['prccm'].shift(-22)
CRSP_comp_merge['forward_twentythree_month_prccm'] = CRSP_comp_merge.groupby('GVKEY')['prccm'].shift(-23)
CRSP_comp_merge['forward_twentyfour_month_prccm'] = CRSP_comp_merge.groupby('GVKEY')['prccm'].shift(-24)
CRSP_comp_merge['forward_twentyfive_month_prccm'] = CRSP_comp_merge.groupby('GVKEY')['prccm'].shift(-25)
CRSP_comp_merge['forward_twentysix_month_prccm'] = CRSP_comp_merge.groupby('GVKEY')['prccm'].shift(-26)
CRSP_comp_merge['forward_twentyseven_month_prccm'] = CRSP_comp_merge.groupby('GVKEY')['prccm'].shift(-27)
CRSP_comp_merge['forward_twentyeight_month_prccm'] = CRSP_comp_merge.groupby('GVKEY')['prccm'].shift(-28)
CRSP_comp_merge['forward_twentynine_month_prccm'] = CRSP_comp_merge.groupby('GVKEY')['prccm'].shift(-29)
CRSP_comp_merge['forward_thirty_month_prccm'] = CRSP_comp_merge.groupby('GVKEY')['prccm'].shift(-30)
CRSP_comp_merge['forward_thirtyone_month_prccm'] = CRSP_comp_merge.groupby('GVKEY')['prccm'].shift(-31)
CRSP_comp_merge['forward_thirtytwo_month_prccm'] = CRSP_comp_merge.groupby('GVKEY')['prccm'].shift(-32)
CRSP_comp_merge['forward_thirtythree_month_prccm'] = CRSP_comp_merge.groupby('GVKEY')['prccm'].shift(-33)
CRSP_comp_merge['forward_thirtyfour_month_prccm'] = CRSP_comp_merge.groupby('GVKEY')['prccm'].shift(-34)
CRSP_comp_merge['forward_thirtyfive_month_prccm'] = CRSP_comp_merge.groupby('GVKEY')['prccm'].shift(-35)
CRSP_comp_merge['forward_thirtysix_month_prccm'] = CRSP_comp_merge.groupby('GVKEY')['prccm'].shift(-36)

In [27]:
# CRSP - Enrich - Forward (AJEXM -- Cumulative Adjustment Factor - Ex Date -Monthly) 1 – 36 months
CRSP_comp_merge['forward_one_month_ajexm'] = CRSP_comp_merge.groupby('GVKEY')['ajexm'].shift(-1)
CRSP_comp_merge['forward_two_month_ajexm'] = CRSP_comp_merge.groupby('GVKEY')['ajexm'].shift(-2)
CRSP_comp_merge['forward_three_month_ajexm'] = CRSP_comp_merge.groupby('GVKEY')['ajexm'].shift(-3)
CRSP_comp_merge['forward_four_month_ajexm'] = CRSP_comp_merge.groupby('GVKEY')['ajexm'].shift(-4)
CRSP_comp_merge['forward_five_month_ajexm'] = CRSP_comp_merge.groupby('GVKEY')['ajexm'].shift(-5)
CRSP_comp_merge['forward_six_month_ajexm'] = CRSP_comp_merge.groupby('GVKEY')['ajexm'].shift(-6)
CRSP_comp_merge['forward_seven_month_ajexm'] = CRSP_comp_merge.groupby('GVKEY')['ajexm'].shift(-7)
CRSP_comp_merge['forward_eight_month_ajexm'] = CRSP_comp_merge.groupby('GVKEY')['ajexm'].shift(-8)
CRSP_comp_merge['forward_nine_month_ajexm'] = CRSP_comp_merge.groupby('GVKEY')['ajexm'].shift(-9)
CRSP_comp_merge['forward_ten_month_ajexm'] = CRSP_comp_merge.groupby('GVKEY')['ajexm'].shift(-10)
CRSP_comp_merge['forward_eleven_month_ajexm'] = CRSP_comp_merge.groupby('GVKEY')['ajexm'].shift(-11)
CRSP_comp_merge['forward_twelve_month_ajexm'] = CRSP_comp_merge.groupby('GVKEY')['ajexm'].shift(-12)
CRSP_comp_merge['forward_thirteen_month_ajexm'] = CRSP_comp_merge.groupby('GVKEY')['ajexm'].shift(-13)
CRSP_comp_merge['forward_fourteen_month_ajexm'] = CRSP_comp_merge.groupby('GVKEY')['ajexm'].shift(-14)
CRSP_comp_merge['forward_fifteen_month_ajexm'] = CRSP_comp_merge.groupby('GVKEY')['ajexm'].shift(-15)
CRSP_comp_merge['forward_sixteen_month_ajexm'] = CRSP_comp_merge.groupby('GVKEY')['ajexm'].shift(-16)
CRSP_comp_merge['forward_seventeen_month_ajexm'] = CRSP_comp_merge.groupby('GVKEY')['ajexm'].shift(-17)
CRSP_comp_merge['forward_eighteen_month_ajexm'] = CRSP_comp_merge.groupby('GVKEY')['ajexm'].shift(-18)
CRSP_comp_merge['forward_nineteen_month_ajexm'] = CRSP_comp_merge.groupby('GVKEY')['ajexm'].shift(-19)
CRSP_comp_merge['forward_twenty_month_ajexm'] = CRSP_comp_merge.groupby('GVKEY')['ajexm'].shift(-20)
CRSP_comp_merge['forward_twentyone_month_ajexm'] = CRSP_comp_merge.groupby('GVKEY')['ajexm'].shift(-21)
CRSP_comp_merge['forward_twentytwo_month_ajexm'] = CRSP_comp_merge.groupby('GVKEY')['ajexm'].shift(-22)
CRSP_comp_merge['forward_twentythree_month_ajexm'] = CRSP_comp_merge.groupby('GVKEY')['ajexm'].shift(-23)
CRSP_comp_merge['forward_twentyfour_month_ajexm'] = CRSP_comp_merge.groupby('GVKEY')['ajexm'].shift(-24)
CRSP_comp_merge['forward_twentyfive_month_ajexm'] = CRSP_comp_merge.groupby('GVKEY')['ajexm'].shift(-25)
CRSP_comp_merge['forward_twentysix_month_ajexm'] = CRSP_comp_merge.groupby('GVKEY')['ajexm'].shift(-26)
CRSP_comp_merge['forward_twentyseven_month_ajexm'] = CRSP_comp_merge.groupby('GVKEY')['ajexm'].shift(-27)
CRSP_comp_merge['forward_twentyeight_month_ajexm'] = CRSP_comp_merge.groupby('GVKEY')['ajexm'].shift(-28)
CRSP_comp_merge['forward_twentynine_month_ajexm'] = CRSP_comp_merge.groupby('GVKEY')['ajexm'].shift(-29)
CRSP_comp_merge['forward_thirty_month_ajexm'] = CRSP_comp_merge.groupby('GVKEY')['ajexm'].shift(-30)
CRSP_comp_merge['forward_thirtyone_month_ajexm'] = CRSP_comp_merge.groupby('GVKEY')['ajexm'].shift(-31)
CRSP_comp_merge['forward_thirtytwo_month_ajexm'] = CRSP_comp_merge.groupby('GVKEY')['ajexm'].shift(-32)
CRSP_comp_merge['forward_thirtythree_month_ajexm'] = CRSP_comp_merge.groupby('GVKEY')['ajexm'].shift(-33)
CRSP_comp_merge['forward_thirtyfour_month_ajexm'] = CRSP_comp_merge.groupby('GVKEY')['ajexm'].shift(-34)
CRSP_comp_merge['forward_thirtyfive_month_ajexm'] = CRSP_comp_merge.groupby('GVKEY')['ajexm'].shift(-35)
CRSP_comp_merge['forward_thirtysix_month_ajexm'] = CRSP_comp_merge.groupby('GVKEY')['ajexm'].shift(-36)

In [28]:
# CRSP - Enrich - Forward (TRFM -- Monthly Total Return Factor) 1 – 36 months
CRSP_comp_merge['forward_one_month_trfm'] = CRSP_comp_merge.groupby('GVKEY')['trfm'].shift(-1)
CRSP_comp_merge['forward_two_month_trfm'] = CRSP_comp_merge.groupby('GVKEY')['trfm'].shift(-2)
CRSP_comp_merge['forward_three_month_trfm'] = CRSP_comp_merge.groupby('GVKEY')['trfm'].shift(-3)
CRSP_comp_merge['forward_four_month_trfm'] = CRSP_comp_merge.groupby('GVKEY')['trfm'].shift(-4)
CRSP_comp_merge['forward_five_month_trfm'] = CRSP_comp_merge.groupby('GVKEY')['trfm'].shift(-5)
CRSP_comp_merge['forward_six_month_trfm'] = CRSP_comp_merge.groupby('GVKEY')['trfm'].shift(-6)
CRSP_comp_merge['forward_seven_month_trfm'] = CRSP_comp_merge.groupby('GVKEY')['trfm'].shift(-7)
CRSP_comp_merge['forward_eight_month_trfm'] = CRSP_comp_merge.groupby('GVKEY')['trfm'].shift(-8)
CRSP_comp_merge['forward_nine_month_trfm'] = CRSP_comp_merge.groupby('GVKEY')['trfm'].shift(-9)
CRSP_comp_merge['forward_ten_month_trfm'] = CRSP_comp_merge.groupby('GVKEY')['trfm'].shift(-10)
CRSP_comp_merge['forward_eleven_month_trfm'] = CRSP_comp_merge.groupby('GVKEY')['trfm'].shift(-11)
CRSP_comp_merge['forward_twelve_month_trfm'] = CRSP_comp_merge.groupby('GVKEY')['trfm'].shift(-12)
CRSP_comp_merge['forward_thirteen_month_trfm'] = CRSP_comp_merge.groupby('GVKEY')['trfm'].shift(-13)
CRSP_comp_merge['forward_fourteen_month_trfm'] = CRSP_comp_merge.groupby('GVKEY')['trfm'].shift(-14)
CRSP_comp_merge['forward_fifteen_month_trfm'] = CRSP_comp_merge.groupby('GVKEY')['trfm'].shift(-15)
CRSP_comp_merge['forward_sixteen_month_trfm'] = CRSP_comp_merge.groupby('GVKEY')['trfm'].shift(-16)
CRSP_comp_merge['forward_seventeen_month_trfm'] = CRSP_comp_merge.groupby('GVKEY')['trfm'].shift(-17)
CRSP_comp_merge['forward_eighteen_month_trfm'] = CRSP_comp_merge.groupby('GVKEY')['trfm'].shift(-18)
CRSP_comp_merge['forward_nineteen_month_trfm'] = CRSP_comp_merge.groupby('GVKEY')['trfm'].shift(-19)
CRSP_comp_merge['forward_twenty_month_trfm'] = CRSP_comp_merge.groupby('GVKEY')['trfm'].shift(-20)
CRSP_comp_merge['forward_twentyone_month_trfm'] = CRSP_comp_merge.groupby('GVKEY')['trfm'].shift(-21)
CRSP_comp_merge['forward_twentytwo_month_trfm'] = CRSP_comp_merge.groupby('GVKEY')['trfm'].shift(-22)
CRSP_comp_merge['forward_twentythree_month_trfm'] = CRSP_comp_merge.groupby('GVKEY')['trfm'].shift(-23)
CRSP_comp_merge['forward_twentyfour_month_trfm'] = CRSP_comp_merge.groupby('GVKEY')['trfm'].shift(-24)
CRSP_comp_merge['forward_twentyfive_month_trfm'] = CRSP_comp_merge.groupby('GVKEY')['trfm'].shift(-25)
CRSP_comp_merge['forward_twentysix_month_trfm'] = CRSP_comp_merge.groupby('GVKEY')['trfm'].shift(-26)
CRSP_comp_merge['forward_twentyseven_month_trfm'] = CRSP_comp_merge.groupby('GVKEY')['trfm'].shift(-27)
CRSP_comp_merge['forward_twentyeight_month_trfm'] = CRSP_comp_merge.groupby('GVKEY')['trfm'].shift(-28)
CRSP_comp_merge['forward_twentynine_month_trfm'] = CRSP_comp_merge.groupby('GVKEY')['trfm'].shift(-29)
CRSP_comp_merge['forward_thirty_month_trfm'] = CRSP_comp_merge.groupby('GVKEY')['trfm'].shift(-30)
CRSP_comp_merge['forward_thirtyone_month_trfm'] = CRSP_comp_merge.groupby('GVKEY')['trfm'].shift(-31)
CRSP_comp_merge['forward_thirtytwo_month_trfm'] = CRSP_comp_merge.groupby('GVKEY')['trfm'].shift(-32)
CRSP_comp_merge['forward_thirtythree_month_trfm'] = CRSP_comp_merge.groupby('GVKEY')['trfm'].shift(-33)
CRSP_comp_merge['forward_thirtyfour_month_trfm'] = CRSP_comp_merge.groupby('GVKEY')['trfm'].shift(-34)
CRSP_comp_merge['forward_thirtyfive_month_trfm'] = CRSP_comp_merge.groupby('GVKEY')['trfm'].shift(-35)
CRSP_comp_merge['forward_thirtysix_month_trfm'] = CRSP_comp_merge.groupby('GVKEY')['trfm'].shift(-36)


In [29]:
# CRSP - Enrich - Forward Return over number of months (1 - 36)
CRSP_comp_merge['forward_one_month_return'] = ((((CRSP_comp_merge['forward_one_month_prccm']/CRSP_comp_merge['forward_one_month_ajexm'])*CRSP_comp_merge['forward_one_month_trfm'])/((CRSP_comp_merge['prccm']/CRSP_comp_merge['ajexm'])*CRSP_comp_merge['trfm']))-1)*100
CRSP_comp_merge['forward_two_month_return'] = ((((CRSP_comp_merge['forward_two_month_prccm']/CRSP_comp_merge['forward_two_month_ajexm'])*CRSP_comp_merge['forward_two_month_trfm'])/((CRSP_comp_merge['prccm']/CRSP_comp_merge['ajexm'])*CRSP_comp_merge['trfm']))-1)*100
CRSP_comp_merge['forward_three_month_return'] = ((((CRSP_comp_merge['forward_three_month_prccm']/CRSP_comp_merge['forward_three_month_ajexm'])*CRSP_comp_merge['forward_three_month_trfm'])/((CRSP_comp_merge['prccm']/CRSP_comp_merge['ajexm'])*CRSP_comp_merge['trfm']))-1)*100
CRSP_comp_merge['forward_four_month_return'] = ((((CRSP_comp_merge['forward_four_month_prccm']/CRSP_comp_merge['forward_four_month_ajexm'])*CRSP_comp_merge['forward_four_month_trfm'])/((CRSP_comp_merge['prccm']/CRSP_comp_merge['ajexm'])*CRSP_comp_merge['trfm']))-1)*100
CRSP_comp_merge['forward_five_month_return'] = ((((CRSP_comp_merge['forward_five_month_prccm']/CRSP_comp_merge['forward_five_month_ajexm'])*CRSP_comp_merge['forward_five_month_trfm'])/((CRSP_comp_merge['prccm']/CRSP_comp_merge['ajexm'])*CRSP_comp_merge['trfm']))-1)*100
CRSP_comp_merge['forward_six_month_return'] = ((((CRSP_comp_merge['forward_six_month_prccm']/CRSP_comp_merge['forward_six_month_ajexm'])*CRSP_comp_merge['forward_six_month_trfm'])/((CRSP_comp_merge['prccm']/CRSP_comp_merge['ajexm'])*CRSP_comp_merge['trfm']))-1)*100
CRSP_comp_merge['forward_seven_month_return'] = ((((CRSP_comp_merge['forward_seven_month_prccm']/CRSP_comp_merge['forward_seven_month_ajexm'])*CRSP_comp_merge['forward_seven_month_trfm'])/((CRSP_comp_merge['prccm']/CRSP_comp_merge['ajexm'])*CRSP_comp_merge['trfm']))-1)*100
CRSP_comp_merge['forward_eight_month_return'] = ((((CRSP_comp_merge['forward_eight_month_prccm']/CRSP_comp_merge['forward_eight_month_ajexm'])*CRSP_comp_merge['forward_eight_month_trfm'])/((CRSP_comp_merge['prccm']/CRSP_comp_merge['ajexm'])*CRSP_comp_merge['trfm']))-1)*100
CRSP_comp_merge['forward_nine_month_return'] = ((((CRSP_comp_merge['forward_nine_month_prccm']/CRSP_comp_merge['forward_nine_month_ajexm'])*CRSP_comp_merge['forward_nine_month_trfm'])/((CRSP_comp_merge['prccm']/CRSP_comp_merge['ajexm'])*CRSP_comp_merge['trfm']))-1)*100
CRSP_comp_merge['forward_ten_month_return'] = ((((CRSP_comp_merge['forward_ten_month_prccm']/CRSP_comp_merge['forward_ten_month_ajexm'])*CRSP_comp_merge['forward_ten_month_trfm'])/((CRSP_comp_merge['prccm']/CRSP_comp_merge['ajexm'])*CRSP_comp_merge['trfm']))-1)*100
CRSP_comp_merge['forward_eleven_month_return'] = ((((CRSP_comp_merge['forward_eleven_month_prccm']/CRSP_comp_merge['forward_eleven_month_ajexm'])*CRSP_comp_merge['forward_eleven_month_trfm'])/((CRSP_comp_merge['prccm']/CRSP_comp_merge['ajexm'])*CRSP_comp_merge['trfm']))-1)*100
CRSP_comp_merge['forward_twelve_month_return'] = ((((CRSP_comp_merge['forward_twelve_month_prccm']/CRSP_comp_merge['forward_twelve_month_ajexm'])*CRSP_comp_merge['forward_twelve_month_trfm'])/((CRSP_comp_merge['prccm']/CRSP_comp_merge['ajexm'])*CRSP_comp_merge['trfm']))-1)*100
CRSP_comp_merge['forward_thirteen_month_return'] = ((((CRSP_comp_merge['forward_thirteen_month_prccm']/CRSP_comp_merge['forward_thirteen_month_ajexm'])*CRSP_comp_merge['forward_thirteen_month_trfm'])/((CRSP_comp_merge['prccm']/CRSP_comp_merge['ajexm'])*CRSP_comp_merge['trfm']))-1)*100
CRSP_comp_merge['forward_fourteen_month_return'] = ((((CRSP_comp_merge['forward_fourteen_month_prccm']/CRSP_comp_merge['forward_fourteen_month_ajexm'])*CRSP_comp_merge['forward_fourteen_month_trfm'])/((CRSP_comp_merge['prccm']/CRSP_comp_merge['ajexm'])*CRSP_comp_merge['trfm']))-1)*100
CRSP_comp_merge['forward_fifteen_month_return'] = ((((CRSP_comp_merge['forward_fifteen_month_prccm']/CRSP_comp_merge['forward_fifteen_month_ajexm'])*CRSP_comp_merge['forward_fifteen_month_trfm'])/((CRSP_comp_merge['prccm']/CRSP_comp_merge['ajexm'])*CRSP_comp_merge['trfm']))-1)*100
CRSP_comp_merge['forward_sixteen_month_return'] = ((((CRSP_comp_merge['forward_sixteen_month_prccm']/CRSP_comp_merge['forward_sixteen_month_ajexm'])*CRSP_comp_merge['forward_sixteen_month_trfm'])/((CRSP_comp_merge['prccm']/CRSP_comp_merge['ajexm'])*CRSP_comp_merge['trfm']))-1)*100
CRSP_comp_merge['forward_seventeen_month_return'] = ((((CRSP_comp_merge['forward_seventeen_month_prccm']/CRSP_comp_merge['forward_seventeen_month_ajexm'])*CRSP_comp_merge['forward_seventeen_month_trfm'])/((CRSP_comp_merge['prccm']/CRSP_comp_merge['ajexm'])*CRSP_comp_merge['trfm']))-1)*100
CRSP_comp_merge['forward_eighteen_month_return'] = ((((CRSP_comp_merge['forward_eighteen_month_prccm']/CRSP_comp_merge['forward_eighteen_month_ajexm'])*CRSP_comp_merge['forward_eighteen_month_trfm'])/((CRSP_comp_merge['prccm']/CRSP_comp_merge['ajexm'])*CRSP_comp_merge['trfm']))-1)*100
CRSP_comp_merge['forward_nineteen_month_return'] = ((((CRSP_comp_merge['forward_nineteen_month_prccm']/CRSP_comp_merge['forward_nineteen_month_ajexm'])*CRSP_comp_merge['forward_nineteen_month_trfm'])/((CRSP_comp_merge['prccm']/CRSP_comp_merge['ajexm'])*CRSP_comp_merge['trfm']))-1)*100
CRSP_comp_merge['forward_twenty_month_return'] = ((((CRSP_comp_merge['forward_twenty_month_prccm']/CRSP_comp_merge['forward_twenty_month_ajexm'])*CRSP_comp_merge['forward_twenty_month_trfm'])/((CRSP_comp_merge['prccm']/CRSP_comp_merge['ajexm'])*CRSP_comp_merge['trfm']))-1)*100
CRSP_comp_merge['forward_twentyone_month_return'] = ((((CRSP_comp_merge['forward_twentyone_month_prccm']/CRSP_comp_merge['forward_twentyone_month_ajexm'])*CRSP_comp_merge['forward_twentyone_month_trfm'])/((CRSP_comp_merge['prccm']/CRSP_comp_merge['ajexm'])*CRSP_comp_merge['trfm']))-1)*100
CRSP_comp_merge['forward_twentytwo_month_return'] = ((((CRSP_comp_merge['forward_twentytwo_month_prccm']/CRSP_comp_merge['forward_twentytwo_month_ajexm'])*CRSP_comp_merge['forward_twentytwo_month_trfm'])/((CRSP_comp_merge['prccm']/CRSP_comp_merge['ajexm'])*CRSP_comp_merge['trfm']))-1)*100
CRSP_comp_merge['forward_twentythree_month_return'] = ((((CRSP_comp_merge['forward_twentythree_month_prccm']/CRSP_comp_merge['forward_twentythree_month_ajexm'])*CRSP_comp_merge['forward_twentythree_month_trfm'])/((CRSP_comp_merge['prccm']/CRSP_comp_merge['ajexm'])*CRSP_comp_merge['trfm']))-1)*100
CRSP_comp_merge['forward_twentyfour_month_return'] = ((((CRSP_comp_merge['forward_twentyfour_month_prccm']/CRSP_comp_merge['forward_twentyfour_month_ajexm'])*CRSP_comp_merge['forward_twentyfour_month_trfm'])/((CRSP_comp_merge['prccm']/CRSP_comp_merge['ajexm'])*CRSP_comp_merge['trfm']))-1)*100
CRSP_comp_merge['forward_twentyfive_month_return'] = ((((CRSP_comp_merge['forward_twentyfive_month_prccm']/CRSP_comp_merge['forward_twentyfive_month_ajexm'])*CRSP_comp_merge['forward_twentyfive_month_trfm'])/((CRSP_comp_merge['prccm']/CRSP_comp_merge['ajexm'])*CRSP_comp_merge['trfm']))-1)*100
CRSP_comp_merge['forward_twentysix_month_return'] = ((((CRSP_comp_merge['forward_twentysix_month_prccm']/CRSP_comp_merge['forward_twentysix_month_ajexm'])*CRSP_comp_merge['forward_twentysix_month_trfm'])/((CRSP_comp_merge['prccm']/CRSP_comp_merge['ajexm'])*CRSP_comp_merge['trfm']))-1)*100
CRSP_comp_merge['forward_twentyseven_month_return'] = ((((CRSP_comp_merge['forward_twentyseven_month_prccm']/CRSP_comp_merge['forward_twentyseven_month_ajexm'])*CRSP_comp_merge['forward_twentyseven_month_trfm'])/((CRSP_comp_merge['prccm']/CRSP_comp_merge['ajexm'])*CRSP_comp_merge['trfm']))-1)*100
CRSP_comp_merge['forward_twentyeight_month_return'] = ((((CRSP_comp_merge['forward_twentyeight_month_prccm']/CRSP_comp_merge['forward_twentyeight_month_ajexm'])*CRSP_comp_merge['forward_twentyeight_month_trfm'])/((CRSP_comp_merge['prccm']/CRSP_comp_merge['ajexm'])*CRSP_comp_merge['trfm']))-1)*100
CRSP_comp_merge['forward_twentynine_month_return'] = ((((CRSP_comp_merge['forward_twentynine_month_prccm']/CRSP_comp_merge['forward_twentynine_month_ajexm'])*CRSP_comp_merge['forward_twentynine_month_trfm'])/((CRSP_comp_merge['prccm']/CRSP_comp_merge['ajexm'])*CRSP_comp_merge['trfm']))-1)*100
CRSP_comp_merge['forward_thirty_month_return'] = ((((CRSP_comp_merge['forward_thirty_month_prccm']/CRSP_comp_merge['forward_thirty_month_ajexm'])*CRSP_comp_merge['forward_thirty_month_trfm'])/((CRSP_comp_merge['prccm']/CRSP_comp_merge['ajexm'])*CRSP_comp_merge['trfm']))-1)*100
CRSP_comp_merge['forward_thirtyone_month_return'] = ((((CRSP_comp_merge['forward_thirtyone_month_prccm']/CRSP_comp_merge['forward_thirtyone_month_ajexm'])*CRSP_comp_merge['forward_thirtyone_month_trfm'])/((CRSP_comp_merge['prccm']/CRSP_comp_merge['ajexm'])*CRSP_comp_merge['trfm']))-1)*100
CRSP_comp_merge['forward_thirtytwo_month_return'] = ((((CRSP_comp_merge['forward_thirtytwo_month_prccm']/CRSP_comp_merge['forward_thirtytwo_month_ajexm'])*CRSP_comp_merge['forward_thirtytwo_month_trfm'])/((CRSP_comp_merge['prccm']/CRSP_comp_merge['ajexm'])*CRSP_comp_merge['trfm']))-1)*100
CRSP_comp_merge['forward_thirtythree_month_return'] = ((((CRSP_comp_merge['forward_thirtythree_month_prccm']/CRSP_comp_merge['forward_thirtythree_month_ajexm'])*CRSP_comp_merge['forward_thirtythree_month_trfm'])/((CRSP_comp_merge['prccm']/CRSP_comp_merge['ajexm'])*CRSP_comp_merge['trfm']))-1)*100
CRSP_comp_merge['forward_thirtyfour_month_return'] = ((((CRSP_comp_merge['forward_thirtyfour_month_prccm']/CRSP_comp_merge['forward_thirtyfour_month_ajexm'])*CRSP_comp_merge['forward_thirtyfour_month_trfm'])/((CRSP_comp_merge['prccm']/CRSP_comp_merge['ajexm'])*CRSP_comp_merge['trfm']))-1)*100
CRSP_comp_merge['forward_thirtyfive_month_return'] = ((((CRSP_comp_merge['forward_thirtyfive_month_prccm']/CRSP_comp_merge['forward_thirtyfive_month_ajexm'])*CRSP_comp_merge['forward_thirtyfive_month_trfm'])/((CRSP_comp_merge['prccm']/CRSP_comp_merge['ajexm'])*CRSP_comp_merge['trfm']))-1)*100
CRSP_comp_merge['forward_thirtysix_month_return'] = ((((CRSP_comp_merge['forward_thirtysix_month_prccm']/CRSP_comp_merge['forward_thirtysix_month_ajexm'])*CRSP_comp_merge['forward_thirtysix_month_trfm'])/((CRSP_comp_merge['prccm']/CRSP_comp_merge['ajexm'])*CRSP_comp_merge['trfm']))-1)*100

In [30]:
# CRSP - Enrich - Past (PRCCM -- Price - Close - Monthly) 1 – 36 months
CRSP_comp_merge['past_one_month_prccm'] = CRSP_comp_merge.groupby('GVKEY')['prccm'].shift(1)
CRSP_comp_merge['past_two_month_prccm'] = CRSP_comp_merge.groupby('GVKEY')['prccm'].shift(2)
CRSP_comp_merge['past_three_month_prccm'] = CRSP_comp_merge.groupby('GVKEY')['prccm'].shift(3)
CRSP_comp_merge['past_four_month_prccm'] = CRSP_comp_merge.groupby('GVKEY')['prccm'].shift(4)
CRSP_comp_merge['past_five_month_prccm'] = CRSP_comp_merge.groupby('GVKEY')['prccm'].shift(5)
CRSP_comp_merge['past_six_month_prccm'] = CRSP_comp_merge.groupby('GVKEY')['prccm'].shift(6)
CRSP_comp_merge['past_seven_month_prccm'] = CRSP_comp_merge.groupby('GVKEY')['prccm'].shift(7)
CRSP_comp_merge['past_eight_month_prccm'] = CRSP_comp_merge.groupby('GVKEY')['prccm'].shift(8)
CRSP_comp_merge['past_nine_month_prccm'] = CRSP_comp_merge.groupby('GVKEY')['prccm'].shift(9)
CRSP_comp_merge['past_ten_month_prccm'] = CRSP_comp_merge.groupby('GVKEY')['prccm'].shift(10)
CRSP_comp_merge['past_eleven_month_prccm'] = CRSP_comp_merge.groupby('GVKEY')['prccm'].shift(11)
CRSP_comp_merge['past_twelve_month_prccm'] = CRSP_comp_merge.groupby('GVKEY')['prccm'].shift(12)
CRSP_comp_merge['past_thirteen_month_prccm'] = CRSP_comp_merge.groupby('GVKEY')['prccm'].shift(13)
CRSP_comp_merge['past_fourteen_month_prccm'] = CRSP_comp_merge.groupby('GVKEY')['prccm'].shift(14)
CRSP_comp_merge['past_fifteen_month_prccm'] = CRSP_comp_merge.groupby('GVKEY')['prccm'].shift(15)
CRSP_comp_merge['past_sixteen_month_prccm'] = CRSP_comp_merge.groupby('GVKEY')['prccm'].shift(16)
CRSP_comp_merge['past_seventeen_month_prccm'] = CRSP_comp_merge.groupby('GVKEY')['prccm'].shift(17)
CRSP_comp_merge['past_eighteen_month_prccm'] = CRSP_comp_merge.groupby('GVKEY')['prccm'].shift(18)
CRSP_comp_merge['past_nineteen_month_prccm'] = CRSP_comp_merge.groupby('GVKEY')['prccm'].shift(19)
CRSP_comp_merge['past_twenty_month_prccm'] = CRSP_comp_merge.groupby('GVKEY')['prccm'].shift(20)
CRSP_comp_merge['past_twentyone_month_prccm'] = CRSP_comp_merge.groupby('GVKEY')['prccm'].shift(21)
CRSP_comp_merge['past_twentytwo_month_prccm'] = CRSP_comp_merge.groupby('GVKEY')['prccm'].shift(22)
CRSP_comp_merge['past_twentythree_month_prccm'] = CRSP_comp_merge.groupby('GVKEY')['prccm'].shift(23)
CRSP_comp_merge['past_twentyfour_month_prccm'] = CRSP_comp_merge.groupby('GVKEY')['prccm'].shift(24)
CRSP_comp_merge['past_twentyfive_month_prccm'] = CRSP_comp_merge.groupby('GVKEY')['prccm'].shift(25)
CRSP_comp_merge['past_twentysix_month_prccm'] = CRSP_comp_merge.groupby('GVKEY')['prccm'].shift(26)
CRSP_comp_merge['past_twentyseven_month_prccm'] = CRSP_comp_merge.groupby('GVKEY')['prccm'].shift(27)
CRSP_comp_merge['past_twentyeight_month_prccm'] = CRSP_comp_merge.groupby('GVKEY')['prccm'].shift(28)
CRSP_comp_merge['past_twentynine_month_prccm'] = CRSP_comp_merge.groupby('GVKEY')['prccm'].shift(29)
CRSP_comp_merge['past_thirty_month_prccm'] = CRSP_comp_merge.groupby('GVKEY')['prccm'].shift(30)
CRSP_comp_merge['past_thirtyone_month_prccm'] = CRSP_comp_merge.groupby('GVKEY')['prccm'].shift(31)
CRSP_comp_merge['past_thirtytwo_month_prccm'] = CRSP_comp_merge.groupby('GVKEY')['prccm'].shift(32)
CRSP_comp_merge['past_thirtythree_month_prccm'] = CRSP_comp_merge.groupby('GVKEY')['prccm'].shift(33)
CRSP_comp_merge['past_thirtyfour_month_prccm'] = CRSP_comp_merge.groupby('GVKEY')['prccm'].shift(34)
CRSP_comp_merge['past_thirtyfive_month_prccm'] = CRSP_comp_merge.groupby('GVKEY')['prccm'].shift(35)
CRSP_comp_merge['past_thirtysix_month_prccm'] = CRSP_comp_merge.groupby('GVKEY')['prccm'].shift(36)

In [31]:
# CRSP - Enrich -  Past (AJEXM -- Cumulative Adjustment Factor - Ex Date -Monthly) 1 – 36 months
CRSP_comp_merge['past_one_month_ajexm'] = CRSP_comp_merge.groupby('GVKEY')['ajexm'].shift(1)
CRSP_comp_merge['past_two_month_ajexm'] = CRSP_comp_merge.groupby('GVKEY')['ajexm'].shift(2)
CRSP_comp_merge['past_three_month_ajexm'] = CRSP_comp_merge.groupby('GVKEY')['ajexm'].shift(3)
CRSP_comp_merge['past_four_month_ajexm'] = CRSP_comp_merge.groupby('GVKEY')['ajexm'].shift(4)
CRSP_comp_merge['past_five_month_ajexm'] = CRSP_comp_merge.groupby('GVKEY')['ajexm'].shift(5)
CRSP_comp_merge['past_six_month_ajexm'] = CRSP_comp_merge.groupby('GVKEY')['ajexm'].shift(6)
CRSP_comp_merge['past_seven_month_ajexm'] = CRSP_comp_merge.groupby('GVKEY')['ajexm'].shift(7)
CRSP_comp_merge['past_eight_month_ajexm'] = CRSP_comp_merge.groupby('GVKEY')['ajexm'].shift(8)
CRSP_comp_merge['past_nine_month_ajexm'] = CRSP_comp_merge.groupby('GVKEY')['ajexm'].shift(9)
CRSP_comp_merge['past_ten_month_ajexm'] = CRSP_comp_merge.groupby('GVKEY')['ajexm'].shift(10)
CRSP_comp_merge['past_eleven_month_ajexm'] = CRSP_comp_merge.groupby('GVKEY')['ajexm'].shift(11)
CRSP_comp_merge['past_twelve_month_ajexm'] = CRSP_comp_merge.groupby('GVKEY')['ajexm'].shift(12)
CRSP_comp_merge['past_thirteen_month_ajexm'] = CRSP_comp_merge.groupby('GVKEY')['ajexm'].shift(13)
CRSP_comp_merge['past_fourteen_month_ajexm'] = CRSP_comp_merge.groupby('GVKEY')['ajexm'].shift(14)
CRSP_comp_merge['past_fifteen_month_ajexm'] = CRSP_comp_merge.groupby('GVKEY')['ajexm'].shift(15)
CRSP_comp_merge['past_sixteen_month_ajexm'] = CRSP_comp_merge.groupby('GVKEY')['ajexm'].shift(16)
CRSP_comp_merge['past_seventeen_month_ajexm'] = CRSP_comp_merge.groupby('GVKEY')['ajexm'].shift(17)
CRSP_comp_merge['past_eighteen_month_ajexm'] = CRSP_comp_merge.groupby('GVKEY')['ajexm'].shift(18)
CRSP_comp_merge['past_nineteen_month_ajexm'] = CRSP_comp_merge.groupby('GVKEY')['ajexm'].shift(19)
CRSP_comp_merge['past_twenty_month_ajexm'] = CRSP_comp_merge.groupby('GVKEY')['ajexm'].shift(20)
CRSP_comp_merge['past_twentyone_month_ajexm'] = CRSP_comp_merge.groupby('GVKEY')['ajexm'].shift(21)
CRSP_comp_merge['past_twentytwo_month_ajexm'] = CRSP_comp_merge.groupby('GVKEY')['ajexm'].shift(22)
CRSP_comp_merge['past_twentythree_month_ajexm'] = CRSP_comp_merge.groupby('GVKEY')['ajexm'].shift(23)
CRSP_comp_merge['past_twentyfour_month_ajexm'] = CRSP_comp_merge.groupby('GVKEY')['ajexm'].shift(24)
CRSP_comp_merge['past_twentyfive_month_ajexm'] = CRSP_comp_merge.groupby('GVKEY')['ajexm'].shift(25)
CRSP_comp_merge['past_twentysix_month_ajexm'] = CRSP_comp_merge.groupby('GVKEY')['ajexm'].shift(26)
CRSP_comp_merge['past_twentyseven_month_ajexm'] = CRSP_comp_merge.groupby('GVKEY')['ajexm'].shift(27)
CRSP_comp_merge['past_twentyeight_month_ajexm'] = CRSP_comp_merge.groupby('GVKEY')['ajexm'].shift(28)
CRSP_comp_merge['past_twentynine_month_ajexm'] = CRSP_comp_merge.groupby('GVKEY')['ajexm'].shift(29)
CRSP_comp_merge['past_thirty_month_ajexm'] = CRSP_comp_merge.groupby('GVKEY')['ajexm'].shift(30)
CRSP_comp_merge['past_thirtyone_month_ajexm'] = CRSP_comp_merge.groupby('GVKEY')['ajexm'].shift(31)
CRSP_comp_merge['past_thirtytwo_month_ajexm'] = CRSP_comp_merge.groupby('GVKEY')['ajexm'].shift(32)
CRSP_comp_merge['past_thirtythree_month_ajexm'] = CRSP_comp_merge.groupby('GVKEY')['ajexm'].shift(33)
CRSP_comp_merge['past_thirtyfour_month_ajexm'] = CRSP_comp_merge.groupby('GVKEY')['ajexm'].shift(34)
CRSP_comp_merge['past_thirtyfive_month_ajexm'] = CRSP_comp_merge.groupby('GVKEY')['ajexm'].shift(35)
CRSP_comp_merge['past_thirtysix_month_ajexm'] = CRSP_comp_merge.groupby('GVKEY')['ajexm'].shift(36)

In [32]:
# CRSP - Enrich -  Past (TRFM -- Monthly Total Return Factor) 1 – 36 months
CRSP_comp_merge['past_one_month_trfm'] = CRSP_comp_merge.groupby('GVKEY')['trfm'].shift(1)
CRSP_comp_merge['past_two_month_trfm'] = CRSP_comp_merge.groupby('GVKEY')['trfm'].shift(2)
CRSP_comp_merge['past_three_month_trfm'] = CRSP_comp_merge.groupby('GVKEY')['trfm'].shift(3)
CRSP_comp_merge['past_four_month_trfm'] = CRSP_comp_merge.groupby('GVKEY')['trfm'].shift(4)
CRSP_comp_merge['past_five_month_trfm'] = CRSP_comp_merge.groupby('GVKEY')['trfm'].shift(5)
CRSP_comp_merge['past_six_month_trfm'] = CRSP_comp_merge.groupby('GVKEY')['trfm'].shift(6)
CRSP_comp_merge['past_seven_month_trfm'] = CRSP_comp_merge.groupby('GVKEY')['trfm'].shift(7)
CRSP_comp_merge['past_eight_month_trfm'] = CRSP_comp_merge.groupby('GVKEY')['trfm'].shift(8)
CRSP_comp_merge['past_nine_month_trfm'] = CRSP_comp_merge.groupby('GVKEY')['trfm'].shift(9)
CRSP_comp_merge['past_ten_month_trfm'] = CRSP_comp_merge.groupby('GVKEY')['trfm'].shift(10)
CRSP_comp_merge['past_eleven_month_trfm'] = CRSP_comp_merge.groupby('GVKEY')['trfm'].shift(11)
CRSP_comp_merge['past_twelve_month_trfm'] = CRSP_comp_merge.groupby('GVKEY')['trfm'].shift(12)
CRSP_comp_merge['past_thirteen_month_trfm'] = CRSP_comp_merge.groupby('GVKEY')['trfm'].shift(13)
CRSP_comp_merge['past_fourteen_month_trfm'] = CRSP_comp_merge.groupby('GVKEY')['trfm'].shift(14)
CRSP_comp_merge['past_fifteen_month_trfm'] = CRSP_comp_merge.groupby('GVKEY')['trfm'].shift(15)
CRSP_comp_merge['past_sixteen_month_trfm'] = CRSP_comp_merge.groupby('GVKEY')['trfm'].shift(16)
CRSP_comp_merge['past_seventeen_month_trfm'] = CRSP_comp_merge.groupby('GVKEY')['trfm'].shift(17)
CRSP_comp_merge['past_eighteen_month_trfm'] = CRSP_comp_merge.groupby('GVKEY')['trfm'].shift(18)
CRSP_comp_merge['past_nineteen_month_trfm'] = CRSP_comp_merge.groupby('GVKEY')['trfm'].shift(19)
CRSP_comp_merge['past_twenty_month_trfm'] = CRSP_comp_merge.groupby('GVKEY')['trfm'].shift(20)
CRSP_comp_merge['past_twentyone_month_trfm'] = CRSP_comp_merge.groupby('GVKEY')['trfm'].shift(21)
CRSP_comp_merge['past_twentytwo_month_trfm'] = CRSP_comp_merge.groupby('GVKEY')['trfm'].shift(22)
CRSP_comp_merge['past_twentythree_month_trfm'] = CRSP_comp_merge.groupby('GVKEY')['trfm'].shift(23)
CRSP_comp_merge['past_twentyfour_month_trfm'] = CRSP_comp_merge.groupby('GVKEY')['trfm'].shift(24)
CRSP_comp_merge['past_twentyfive_month_trfm'] = CRSP_comp_merge.groupby('GVKEY')['trfm'].shift(25)
CRSP_comp_merge['past_twentysix_month_trfm'] = CRSP_comp_merge.groupby('GVKEY')['trfm'].shift(26)
CRSP_comp_merge['past_twentyseven_month_trfm'] = CRSP_comp_merge.groupby('GVKEY')['trfm'].shift(27)
CRSP_comp_merge['past_twentyeight_month_trfm'] = CRSP_comp_merge.groupby('GVKEY')['trfm'].shift(28)
CRSP_comp_merge['past_twentynine_month_trfm'] = CRSP_comp_merge.groupby('GVKEY')['trfm'].shift(29)
CRSP_comp_merge['past_thirty_month_trfm'] = CRSP_comp_merge.groupby('GVKEY')['trfm'].shift(30)
CRSP_comp_merge['past_thirtyone_month_trfm'] = CRSP_comp_merge.groupby('GVKEY')['trfm'].shift(31)
CRSP_comp_merge['past_thirtytwo_month_trfm'] = CRSP_comp_merge.groupby('GVKEY')['trfm'].shift(32)
CRSP_comp_merge['past_thirtythree_month_trfm'] = CRSP_comp_merge.groupby('GVKEY')['trfm'].shift(33)
CRSP_comp_merge['past_thirtyfour_month_trfm'] = CRSP_comp_merge.groupby('GVKEY')['trfm'].shift(34)
CRSP_comp_merge['past_thirtyfive_month_trfm'] = CRSP_comp_merge.groupby('GVKEY')['trfm'].shift(35)
CRSP_comp_merge['past_thirtysix_month_trfm'] = CRSP_comp_merge.groupby('GVKEY')['trfm'].shift(36)

In [33]:
# CRSP - Enrich -  Past Return over number of months (1 - 36)
CRSP_comp_merge['past_one_month_return'] = ((((CRSP_comp_merge['prccm']/CRSP_comp_merge['ajexm'])*CRSP_comp_merge['past_one_month_trfm'])/((CRSP_comp_merge['past_one_month_prccm']/CRSP_comp_merge['past_one_month_ajexm'])*CRSP_comp_merge['past_one_month_trfm']))-1)*100
CRSP_comp_merge['past_two_month_return'] = ((((CRSP_comp_merge['prccm']/CRSP_comp_merge['ajexm'])*CRSP_comp_merge['past_two_month_trfm'])/((CRSP_comp_merge['past_two_month_prccm']/CRSP_comp_merge['past_two_month_ajexm'])*CRSP_comp_merge['past_two_month_trfm']))-1)*100
CRSP_comp_merge['past_three_month_return'] = ((((CRSP_comp_merge['prccm']/CRSP_comp_merge['ajexm'])*CRSP_comp_merge['past_three_month_trfm'])/((CRSP_comp_merge['past_three_month_prccm']/CRSP_comp_merge['past_three_month_ajexm'])*CRSP_comp_merge['past_three_month_trfm']))-1)*100
CRSP_comp_merge['past_four_month_return'] = ((((CRSP_comp_merge['prccm']/CRSP_comp_merge['ajexm'])*CRSP_comp_merge['past_four_month_trfm'])/((CRSP_comp_merge['past_four_month_prccm']/CRSP_comp_merge['past_four_month_ajexm'])*CRSP_comp_merge['past_four_month_trfm']))-1)*100
CRSP_comp_merge['past_five_month_return'] = ((((CRSP_comp_merge['prccm']/CRSP_comp_merge['ajexm'])*CRSP_comp_merge['past_five_month_trfm'])/((CRSP_comp_merge['past_five_month_prccm']/CRSP_comp_merge['past_five_month_ajexm'])*CRSP_comp_merge['past_five_month_trfm']))-1)*100
CRSP_comp_merge['past_six_month_return'] = ((((CRSP_comp_merge['prccm']/CRSP_comp_merge['ajexm'])*CRSP_comp_merge['past_six_month_trfm'])/((CRSP_comp_merge['past_six_month_prccm']/CRSP_comp_merge['past_six_month_ajexm'])*CRSP_comp_merge['past_six_month_trfm']))-1)*100
CRSP_comp_merge['past_seven_month_return'] = ((((CRSP_comp_merge['prccm']/CRSP_comp_merge['ajexm'])*CRSP_comp_merge['past_seven_month_trfm'])/((CRSP_comp_merge['past_seven_month_prccm']/CRSP_comp_merge['past_seven_month_ajexm'])*CRSP_comp_merge['past_seven_month_trfm']))-1)*100
CRSP_comp_merge['past_eight_month_return'] = ((((CRSP_comp_merge['prccm']/CRSP_comp_merge['ajexm'])*CRSP_comp_merge['past_eight_month_trfm'])/((CRSP_comp_merge['past_eight_month_prccm']/CRSP_comp_merge['past_eight_month_ajexm'])*CRSP_comp_merge['past_eight_month_trfm']))-1)*100
CRSP_comp_merge['past_nine_month_return'] = ((((CRSP_comp_merge['prccm']/CRSP_comp_merge['ajexm'])*CRSP_comp_merge['past_nine_month_trfm'])/((CRSP_comp_merge['past_nine_month_prccm']/CRSP_comp_merge['past_nine_month_ajexm'])*CRSP_comp_merge['past_nine_month_trfm']))-1)*100
CRSP_comp_merge['past_ten_month_return'] = ((((CRSP_comp_merge['prccm']/CRSP_comp_merge['ajexm'])*CRSP_comp_merge['past_ten_month_trfm'])/((CRSP_comp_merge['past_ten_month_prccm']/CRSP_comp_merge['past_ten_month_ajexm'])*CRSP_comp_merge['past_ten_month_trfm']))-1)*100
CRSP_comp_merge['past_eleven_month_return'] = ((((CRSP_comp_merge['prccm']/CRSP_comp_merge['ajexm'])*CRSP_comp_merge['past_eleven_month_trfm'])/((CRSP_comp_merge['past_eleven_month_prccm']/CRSP_comp_merge['past_eleven_month_ajexm'])*CRSP_comp_merge['past_eleven_month_trfm']))-1)*100
CRSP_comp_merge['past_twelve_month_return'] = ((((CRSP_comp_merge['prccm']/CRSP_comp_merge['ajexm'])*CRSP_comp_merge['past_twelve_month_trfm'])/((CRSP_comp_merge['past_twelve_month_prccm']/CRSP_comp_merge['past_twelve_month_ajexm'])*CRSP_comp_merge['past_twelve_month_trfm']))-1)*100
CRSP_comp_merge['past_thirteen_month_return'] = ((((CRSP_comp_merge['prccm']/CRSP_comp_merge['ajexm'])*CRSP_comp_merge['past_thirteen_month_trfm'])/((CRSP_comp_merge['past_thirteen_month_prccm']/CRSP_comp_merge['past_thirteen_month_ajexm'])*CRSP_comp_merge['past_thirteen_month_trfm']))-1)*100
CRSP_comp_merge['past_fourteen_month_return'] = ((((CRSP_comp_merge['prccm']/CRSP_comp_merge['ajexm'])*CRSP_comp_merge['past_fourteen_month_trfm'])/((CRSP_comp_merge['past_fourteen_month_prccm']/CRSP_comp_merge['past_fourteen_month_ajexm'])*CRSP_comp_merge['past_fourteen_month_trfm']))-1)*100
CRSP_comp_merge['past_fifteen_month_return'] = ((((CRSP_comp_merge['prccm']/CRSP_comp_merge['ajexm'])*CRSP_comp_merge['past_fifteen_month_trfm'])/((CRSP_comp_merge['past_fifteen_month_prccm']/CRSP_comp_merge['past_fifteen_month_ajexm'])*CRSP_comp_merge['past_fifteen_month_trfm']))-1)*100
CRSP_comp_merge['past_sixteen_month_return'] = ((((CRSP_comp_merge['prccm']/CRSP_comp_merge['ajexm'])*CRSP_comp_merge['past_sixteen_month_trfm'])/((CRSP_comp_merge['past_sixteen_month_prccm']/CRSP_comp_merge['past_sixteen_month_ajexm'])*CRSP_comp_merge['past_sixteen_month_trfm']))-1)*100
CRSP_comp_merge['past_seventeen_month_return'] = ((((CRSP_comp_merge['prccm']/CRSP_comp_merge['ajexm'])*CRSP_comp_merge['past_seventeen_month_trfm'])/((CRSP_comp_merge['past_seventeen_month_prccm']/CRSP_comp_merge['past_seventeen_month_ajexm'])*CRSP_comp_merge['past_seventeen_month_trfm']))-1)*100
CRSP_comp_merge['past_eighteen_month_return'] = ((((CRSP_comp_merge['prccm']/CRSP_comp_merge['ajexm'])*CRSP_comp_merge['past_eighteen_month_trfm'])/((CRSP_comp_merge['past_eighteen_month_prccm']/CRSP_comp_merge['past_eighteen_month_ajexm'])*CRSP_comp_merge['past_eighteen_month_trfm']))-1)*100
CRSP_comp_merge['past_nineteen_month_return'] = ((((CRSP_comp_merge['prccm']/CRSP_comp_merge['ajexm'])*CRSP_comp_merge['past_nineteen_month_trfm'])/((CRSP_comp_merge['past_nineteen_month_prccm']/CRSP_comp_merge['past_nineteen_month_ajexm'])*CRSP_comp_merge['past_nineteen_month_trfm']))-1)*100
CRSP_comp_merge['past_twenty_month_return'] = ((((CRSP_comp_merge['prccm']/CRSP_comp_merge['ajexm'])*CRSP_comp_merge['past_twenty_month_trfm'])/((CRSP_comp_merge['past_twenty_month_prccm']/CRSP_comp_merge['past_twenty_month_ajexm'])*CRSP_comp_merge['past_twenty_month_trfm']))-1)*100
CRSP_comp_merge['past_twentyone_month_return'] = ((((CRSP_comp_merge['prccm']/CRSP_comp_merge['ajexm'])*CRSP_comp_merge['past_twentyone_month_trfm'])/((CRSP_comp_merge['past_twentyone_month_prccm']/CRSP_comp_merge['past_twentyone_month_ajexm'])*CRSP_comp_merge['past_twentyone_month_trfm']))-1)*100
CRSP_comp_merge['past_twentytwo_month_return'] = ((((CRSP_comp_merge['prccm']/CRSP_comp_merge['ajexm'])*CRSP_comp_merge['past_twentytwo_month_trfm'])/((CRSP_comp_merge['past_twentytwo_month_prccm']/CRSP_comp_merge['past_twentytwo_month_ajexm'])*CRSP_comp_merge['past_twentytwo_month_trfm']))-1)*100
CRSP_comp_merge['past_twentythree_month_return'] = ((((CRSP_comp_merge['prccm']/CRSP_comp_merge['ajexm'])*CRSP_comp_merge['past_twentythree_month_trfm'])/((CRSP_comp_merge['past_twentythree_month_prccm']/CRSP_comp_merge['past_twentythree_month_ajexm'])*CRSP_comp_merge['past_twentythree_month_trfm']))-1)*100
CRSP_comp_merge['past_twentyfour_month_return'] = ((((CRSP_comp_merge['prccm']/CRSP_comp_merge['ajexm'])*CRSP_comp_merge['past_twentyfour_month_trfm'])/((CRSP_comp_merge['past_twentyfour_month_prccm']/CRSP_comp_merge['past_twentyfour_month_ajexm'])*CRSP_comp_merge['past_twentyfour_month_trfm']))-1)*100
CRSP_comp_merge['past_twentyfive_month_return'] = ((((CRSP_comp_merge['prccm']/CRSP_comp_merge['ajexm'])*CRSP_comp_merge['past_twentyfive_month_trfm'])/((CRSP_comp_merge['past_twentyfive_month_prccm']/CRSP_comp_merge['past_twentyfive_month_ajexm'])*CRSP_comp_merge['past_twentyfive_month_trfm']))-1)*100
CRSP_comp_merge['past_twentysix_month_return'] = ((((CRSP_comp_merge['prccm']/CRSP_comp_merge['ajexm'])*CRSP_comp_merge['past_twentysix_month_trfm'])/((CRSP_comp_merge['past_twentysix_month_prccm']/CRSP_comp_merge['past_twentysix_month_ajexm'])*CRSP_comp_merge['past_twentysix_month_trfm']))-1)*100
CRSP_comp_merge['past_twentyseven_month_return'] = ((((CRSP_comp_merge['prccm']/CRSP_comp_merge['ajexm'])*CRSP_comp_merge['past_twentyseven_month_trfm'])/((CRSP_comp_merge['past_twentyseven_month_prccm']/CRSP_comp_merge['past_twentyseven_month_ajexm'])*CRSP_comp_merge['past_twentyseven_month_trfm']))-1)*100
CRSP_comp_merge['past_twentyeight_month_return'] = ((((CRSP_comp_merge['prccm']/CRSP_comp_merge['ajexm'])*CRSP_comp_merge['past_twentyeight_month_trfm'])/((CRSP_comp_merge['past_twentyeight_month_prccm']/CRSP_comp_merge['past_twentyeight_month_ajexm'])*CRSP_comp_merge['past_twentyeight_month_trfm']))-1)*100
CRSP_comp_merge['past_twentynine_month_return'] = ((((CRSP_comp_merge['prccm']/CRSP_comp_merge['ajexm'])*CRSP_comp_merge['past_twentynine_month_trfm'])/((CRSP_comp_merge['past_twentynine_month_prccm']/CRSP_comp_merge['past_twentynine_month_ajexm'])*CRSP_comp_merge['past_twentynine_month_trfm']))-1)*100
CRSP_comp_merge['past_thirty_month_return'] = ((((CRSP_comp_merge['prccm']/CRSP_comp_merge['ajexm'])*CRSP_comp_merge['past_thirty_month_trfm'])/((CRSP_comp_merge['past_thirty_month_prccm']/CRSP_comp_merge['past_thirty_month_ajexm'])*CRSP_comp_merge['past_thirty_month_trfm']))-1)*100
CRSP_comp_merge['past_thirtyone_month_return'] = ((((CRSP_comp_merge['prccm']/CRSP_comp_merge['ajexm'])*CRSP_comp_merge['past_thirtyone_month_trfm'])/((CRSP_comp_merge['past_thirtyone_month_prccm']/CRSP_comp_merge['past_thirtyone_month_ajexm'])*CRSP_comp_merge['past_thirtyone_month_trfm']))-1)*100
CRSP_comp_merge['past_thirtytwo_month_return'] = ((((CRSP_comp_merge['prccm']/CRSP_comp_merge['ajexm'])*CRSP_comp_merge['past_thirtytwo_month_trfm'])/((CRSP_comp_merge['past_thirtytwo_month_prccm']/CRSP_comp_merge['past_thirtytwo_month_ajexm'])*CRSP_comp_merge['past_thirtytwo_month_trfm']))-1)*100
CRSP_comp_merge['past_thirtythree_month_return'] = ((((CRSP_comp_merge['prccm']/CRSP_comp_merge['ajexm'])*CRSP_comp_merge['past_thirtythree_month_trfm'])/((CRSP_comp_merge['past_thirtythree_month_prccm']/CRSP_comp_merge['past_thirtythree_month_ajexm'])*CRSP_comp_merge['past_thirtythree_month_trfm']))-1)*100
CRSP_comp_merge['past_thirtyfour_month_return'] = ((((CRSP_comp_merge['prccm']/CRSP_comp_merge['ajexm'])*CRSP_comp_merge['past_thirtyfour_month_trfm'])/((CRSP_comp_merge['past_thirtyfour_month_prccm']/CRSP_comp_merge['past_thirtyfour_month_ajexm'])*CRSP_comp_merge['past_thirtyfour_month_trfm']))-1)*100
CRSP_comp_merge['past_thirtyfive_month_return'] = ((((CRSP_comp_merge['prccm']/CRSP_comp_merge['ajexm'])*CRSP_comp_merge['past_thirtyfive_month_trfm'])/((CRSP_comp_merge['past_thirtyfive_month_prccm']/CRSP_comp_merge['past_thirtyfive_month_ajexm'])*CRSP_comp_merge['past_thirtyfive_month_trfm']))-1)*100
CRSP_comp_merge['past_thirtysix_month_return'] = ((((CRSP_comp_merge['prccm']/CRSP_comp_merge['ajexm'])*CRSP_comp_merge['past_thirtysix_month_trfm'])/((CRSP_comp_merge['past_thirtysix_month_prccm']/CRSP_comp_merge['past_thirtysix_month_ajexm'])*CRSP_comp_merge['past_thirtysix_month_trfm']))-1)*100

In [34]:
# Merge - df & CRSP
df = df.merge(CRSP_comp_merge, on='GVKEY-year-month', how='left', suffixes=('', '_y'))
# Release Memory
del CRSP_comp_merge

In [35]:
# df - Clean - Remove duplicate features
drop_dups(df)

In [36]:
# Recommendations - Load
# Summary Statistics (Consensus Recommendations) from Wharton
recommendations = pd.read_csv('c:/users/sconner/Desktop/W205_Final/sample_data/Sample_Recommendations_Summary_Statistics.csv', low_memory=False)

In [37]:
# Recommendations - Enrich - Time Features
recommendations['STATPERS'] = pd.to_datetime(recommendations['STATPERS'])
# Create year
recommendations['year'] = recommendations['STATPERS'].dt.year
# Create month
recommendations['month'] = recommendations['STATPERS'].dt.month
# Creat year-month
recommendations["year-month"] = recommendations['STATPERS'].apply(lambda x: x.strftime('%Y-%m'))

In [38]:
# Recommendations - Enrich - Analyst Change
recommendations["recup"] = recommendations["NUMUP"] / recommendations["NUMREC"] 
recommendations["recdown"] = recommendations["NUMDOWN"] / recommendations["NUMREC"]

In [39]:
# Recommendations - Enrich - Time-based Unique Identifiers
recommendations["TIC-year-month"] = recommendations["OFTIC"].map(str) + "-" + recommendations["year-month"]

In [40]:
# Merge - df & Recommendations
df = df.merge(recommendations, on='TIC-year-month', how='left', suffixes=('', '_y'))
# Release Memory
del recommendations

In [41]:
# df - Clean - Remove duplicate features
drop_dups(df)

In [42]:
# Beta Suite - Load
beta_suite = pd.read_csv('c:/users/sconner/Desktop/W205_Final/sample_data/Sample_Beta_Suite.csv', low_memory=False)

In [43]:
# Beta Suite - Enrich - Time Features
# Convert to data-time
beta_suite['DATE'] = pd.to_datetime(beta_suite['DATE'])
# Create year
beta_suite['year'] = beta_suite['DATE'].dt.year
# Create month
beta_suite['month'] = beta_suite['DATE'].dt.month
# Create year-month
beta_suite["year-month"] = beta_suite['DATE'].apply(lambda x: x.strftime('%Y-%m'))

In [44]:
# Beta Suite - Enrich - Time-based Unique Identifiers
beta_suite["PERMNO-year-month"] = beta_suite["PERMNO"].map(str) + "-" + beta_suite["year-month"]

In [45]:
# Merge - df & Beta Suite
df = df.merge(beta_suite, on='PERMNO-year-month', how='left', suffixes=('', '_y'))
# Release Memory
del beta_suite

In [46]:
# df - Clean - Remove duplicate features
drop_dups(df)

In [47]:
# Sector - Enrich
# https://en.wikipedia.org/wiki/Global_Industry_Classification_Standard

sector = pd.DataFrame()

sector['GSECTOR'] = [10.0, 
                       15.0, 
                       20.0, 
                       25.0, 
                       30.0, 
                       35.0, 
                       40.0, 
                       45.0, 
                       50.0, 
                       55.0, 
                       60.0]

sector['sector'] = ['Energy', 
                        'Materials', 
                        'Industrials', 
                        'Consumer Discretionary', 
                        'Consumer Staples', 
                        'Health Care', 
                        'Financials', 
                        'Information Technology', 
                        'Telecommunication Services', 
                        'Utilities', 
                        'Real Estate']

In [48]:
# Merge - df & Sector
df = df.merge(sector, on='GSECTOR', how='left', suffixes=('', '_y'))
# Release Memory
del sector

In [49]:
# df - Clean - Remove duplicate features
drop_dups(df)

In [50]:
# df - Enrich - Time Features
df['january'] = np.where(df['month'] == 1, int(1), int(0))
df['february'] = np.where(df['month'] == 2, int(1), int(0))
df['march'] = np.where(df['month'] == 3, int(1), int(0))
df['april'] = np.where(df['month'] == 4, int(1), int(0))
df['may'] = np.where(df['month'] == 5, int(1), int(0))
df['june'] = np.where(df['month'] == 6, int(1), int(0))
df['july'] = np.where(df['month'] == 7, int(1), int(0))
df['august'] = np.where(df['month'] == 8, int(1), int(0))
df['september'] = np.where(df['month'] == 9, int(1), int(0))
df['october'] = np.where(df['month'] == 10, int(1), int(0))
df['november'] = np.where(df['month'] == 11, int(1), int(0))
df['december'] = np.where(df['month'] == 12, int(1), int(0))

In [51]:
# df - Subset - Relevant features with adequate data

df = df[[
        'GVKEY', 
        'sector',
        'year-month', 
        'forward_one_month_return', 
        'forward_two_month_return', 
        'forward_three_month_return', 
        'forward_four_month_return', 
        'forward_five_month_return', 
        'forward_six_month_return', 
        'forward_seven_month_return', 
        'forward_eight_month_return', 
        'forward_nine_month_return', 
        'forward_ten_month_return', 
        'forward_eleven_month_return', 
        'forward_twelve_month_return', 
        'forward_thirteen_month_return', 
        'forward_fourteen_month_return', 
        'forward_fifteen_month_return', 
        'forward_sixteen_month_return', 
        'forward_seventeen_month_return', 
        'forward_eighteen_month_return', 
        'forward_nineteen_month_return', 
        'forward_twenty_month_return', 
        'forward_twentyone_month_return', 
        'forward_twentytwo_month_return', 
        'forward_twentythree_month_return', 
        'forward_twentyfour_month_return', 
        'forward_twentyfive_month_return', 
        'forward_twentysix_month_return', 
        'forward_twentyseven_month_return', 
        'forward_twentyeight_month_return', 
        'forward_twentynine_month_return', 
        'forward_thirty_month_return', 
        'forward_thirtyone_month_return', 
        'forward_thirtytwo_month_return', 
        'forward_thirtythree_month_return', 
        'forward_thirtyfour_month_return', 
        'forward_thirtyfive_month_return', 
        'forward_thirtysix_month_return', 
        'past_one_month_return', 
        'past_two_month_return', 
        'past_three_month_return', 
        'past_four_month_return', 
        'past_five_month_return', 
        'past_six_month_return', 
        'past_seven_month_return', 
        'past_eight_month_return', 
        'past_nine_month_return', 
        'past_ten_month_return', 
        'past_eleven_month_return', 
        'past_twelve_month_return', 
        'past_thirteen_month_return', 
        'past_fourteen_month_return', 
        'past_fifteen_month_return', 
        'past_sixteen_month_return', 
        'past_seventeen_month_return', 
        'past_eighteen_month_return', 
        'past_nineteen_month_return', 
        'past_twenty_month_return', 
        'past_twentyone_month_return', 
        'past_twentytwo_month_return', 
        'past_twentythree_month_return', 
        'past_twentyfour_month_return', 
        'past_twentyfive_month_return', 
        'past_twentysix_month_return', 
        'past_twentyseven_month_return', 
        'past_twentyeight_month_return', 
        'past_twentynine_month_return', 
        'past_thirty_month_return', 
        'past_thirtyone_month_return', 
        'past_thirtytwo_month_return', 
        'past_thirtythree_month_return', 
        'past_thirtyfour_month_return', 
        'past_thirtyfive_month_return', 
        'past_thirtysix_month_return', 
        'accrual', 
        'adv_sale', 
        'aftret_eq', 
        'aftret_equity', 
        'aftret_invcapx', 
        'at_turn', 
        'bm', 
        'CAPEI', 
        'capital_ratio', 
        'cash_conversion', 
        'cash_debt', 
        'cash_lt', 
        'cash_ratio', 
        'cfm', 
        'curr_debt', 
        'curr_ratio', 
        'de_ratio', 
        'debt_assets', 
        'debt_at', 
        'debt_capital', 
        'debt_ebitda', 
        'debt_invcap', 
        'DIVYIELD', 
        'dltt_be', 
        'dpr', 
        'efftax', 
        'equity_invcap', 
        'evm', 
        'fcf_ocf', 
        'gpm', 
        'GProf', 
        'int_debt', 
        'int_totdebt', 
        'intcov', 
        'intcov_ratio', 
        'inv_turn', 
        'invt_act', 
        'lt_debt', 
        'lt_ppent', 
        'npm', 
        'ocf_lct', 
        'opmad', 
        'opmbd', 
        'pay_turn', 
        'pcf', 
        'pe_exi', 
        'pe_inc', 
        'pe_op_basic', 
        'pe_op_dil', 
        'PEG_1yrforward', 
        'PEG_ltgforward', 
        'PEG_trailing', 
        'pretret_earnat', 
        'pretret_noa', 
        'profit_lct', 
        'ps', 
        'ptb', 
        'ptpm', 
        'quick_ratio', 
        'rd_sale', 
        'rect_act', 
        'rect_turn', 
        'roa', 
        'roce', 
        'roe', 
        'sale_equity', 
        'sale_invcap', 
        'sale_nwc', 
        'short_debt', 
        'staff_sale', 
        'totdebt_invcap', 
        'dvpspm', 
        'dvpsxm', 
        'dvrate', 
        'spcsrc', 
        'alpha', 
        'b_hml', 
        'b_mkt', 
        'b_smb', 
        'b_umd', 
        'exret', 
        'ivol', 
        'n', 
        'R2', 
        'tvol', 
        'BUYPCT', 
        'HOLDPCT',
        'SELLPCT', 
        'MEANREC', 
        'MEDREC',
        'recup', 
        'recdown',
        'STDEV', 
        'january', 
        'february', 
        'march', 
        'april', 
        'may', 
        'june', 
        'july', 
        'august', 
        'september', 
        'october', 
        'november', 
        'december'
        ]]

In [52]:
# # df - Clean - Dividends - DIVYIELD, dvpspm, dvpsxm, & dvrate
# # Companies who do not report dividends are unlikely to have paid dividends, the majority of companies do not pay as well.
df['DIVYIELD'] = df['DIVYIELD'].replace('%','',regex=True).astype('float')/100
df['DIVYIELD'] = pd.to_numeric(df['DIVYIELD'])
df['DIVYIELD'] = df['DIVYIELD'].fillna(0)

df['dvpspm'] = df['dvpspm'].fillna(0)
df['dvpspm'] = pd.to_numeric(df['dvpspm'])
df['dvpsxm'] = df['dvpsxm'].fillna(0)
df['dvpsxm'] = pd.to_numeric(df['dvpsxm'])
df['dvrate'] = df['dvrate'].fillna(0)
df['dvrate'] = pd.to_numeric(df['dvrate'])

In [53]:
# df - Clean - Ratings - SPCSRC
# Per S&P Quality Rankings information; 
# A+ - 98-100 percentile (Highest, 2%)
# A - 92-98 percentile (High, 6%) 
# A- - 86-92 percentile (Above Average, 6%) 
# B+ - 70-86 percentile (Average, 16%) 
# B - 51-70 percentile (Below Average, 19%)
# B- - 25-51 percentile (Lower, 26%)
# C - 1-25 percentile (Lowest, 24%)
# D - 0-1 percentile (In Reorganization, 1%) 

# Using the average of percentile ranges above

df['spcsrc'] = df['spcsrc'].map({
                                'A+': 99,
                                'A': 95,
                                'A-': 89,
                                'B+': 79.5,
                                'B': 60.5,
                                'B-': 38,
                                'C': 13,
                                'D': 0.5,
                                })

df['spcsrc'] = pd.to_numeric(df['spcsrc'])

In [54]:
# df - Clean - Beta Suite - exret, ivol, R2, tvol

df['exret'] = df['exret'].replace('%','',regex=True).astype('float')/100
df['exret'] = pd.to_numeric(df['exret'])

df['ivol'] = df['ivol'].replace('%','',regex=True).astype('float')/100
df['ivol'] = pd.to_numeric(df['ivol'])

df['R2'] = df['R2'].replace('%','',regex=True).astype('float')/100
df['R2'] = pd.to_numeric(df['R2'])

df['tvol'] = df['tvol'].replace('%','',regex=True).astype('float')/100
df['tvol'] = pd.to_numeric(df['tvol'])

In [55]:
# df - Enrich - Market Z-Score
df['past_one_month_return_zscore'] = df.groupby(['year-month'])[['past_one_month_return']].apply(modified_z)
df['past_two_month_return_zscore'] = df.groupby(['year-month'])[['past_two_month_return']].apply(modified_z)
df['past_three_month_return_zscore'] = df.groupby(['year-month'])[['past_three_month_return']].apply(modified_z)
df['past_four_month_return_zscore'] = df.groupby(['year-month'])[['past_four_month_return']].apply(modified_z)
df['past_five_month_return_zscore'] = df.groupby(['year-month'])[['past_five_month_return']].apply(modified_z)
df['past_six_month_return_zscore'] = df.groupby(['year-month'])[['past_six_month_return']].apply(modified_z)
df['past_seven_month_return_zscore'] = df.groupby(['year-month'])[['past_seven_month_return']].apply(modified_z)
df['past_eight_month_return_zscore'] = df.groupby(['year-month'])[['past_eight_month_return']].apply(modified_z)
df['past_nine_month_return_zscore'] = df.groupby(['year-month'])[['past_nine_month_return']].apply(modified_z)
df['past_ten_month_return_zscore'] = df.groupby(['year-month'])[['past_ten_month_return']].apply(modified_z)
df['past_eleven_month_return_zscore'] = df.groupby(['year-month'])[['past_eleven_month_return']].apply(modified_z)
df['past_twelve_month_return_zscore'] = df.groupby(['year-month'])[['past_twelve_month_return']].apply(modified_z)
df['past_thirteen_month_return_zscore'] = df.groupby(['year-month'])[['past_thirteen_month_return']].apply(modified_z)
df['past_fourteen_month_return_zscore'] = df.groupby(['year-month'])[['past_fourteen_month_return']].apply(modified_z)
df['past_fifteen_month_return_zscore'] = df.groupby(['year-month'])[['past_fifteen_month_return']].apply(modified_z)
df['past_sixteen_month_return_zscore'] = df.groupby(['year-month'])[['past_sixteen_month_return']].apply(modified_z)
df['past_seventeen_month_return_zscore'] = df.groupby(['year-month'])[['past_seventeen_month_return']].apply(modified_z)
df['past_eighteen_month_return_zscore'] = df.groupby(['year-month'])[['past_eighteen_month_return']].apply(modified_z)
df['past_nineteen_month_return_zscore'] = df.groupby(['year-month'])[['past_nineteen_month_return']].apply(modified_z)
df['past_twenty_month_return_zscore'] = df.groupby(['year-month'])[['past_twenty_month_return']].apply(modified_z)
df['past_twentyone_month_return_zscore'] = df.groupby(['year-month'])[['past_twentyone_month_return']].apply(modified_z)
df['past_twentytwo_month_return_zscore'] = df.groupby(['year-month'])[['past_twentytwo_month_return']].apply(modified_z)
df['past_twentythree_month_return_zscore'] = df.groupby(['year-month'])[['past_twentythree_month_return']].apply(modified_z)
df['past_twentyfour_month_return_zscore'] = df.groupby(['year-month'])[['past_twentyfour_month_return']].apply(modified_z)
df['past_twentyfive_month_return_zscore'] = df.groupby(['year-month'])[['past_twentyfive_month_return']].apply(modified_z)
df['past_twentysix_month_return_zscore'] = df.groupby(['year-month'])[['past_twentysix_month_return']].apply(modified_z)
df['past_twentyseven_month_return_zscore'] = df.groupby(['year-month'])[['past_twentyseven_month_return']].apply(modified_z)
df['past_twentyeight_month_return_zscore'] = df.groupby(['year-month'])[['past_twentyeight_month_return']].apply(modified_z)
df['past_twentynine_month_return_zscore'] = df.groupby(['year-month'])[['past_twentynine_month_return']].apply(modified_z)
df['past_thirty_month_return_zscore'] = df.groupby(['year-month'])[['past_thirty_month_return']].apply(modified_z)
df['past_thirtyone_month_return_zscore'] = df.groupby(['year-month'])[['past_thirtyone_month_return']].apply(modified_z)
df['past_thirtytwo_month_return_zscore'] = df.groupby(['year-month'])[['past_thirtytwo_month_return']].apply(modified_z)
df['past_thirtythree_month_return_zscore'] = df.groupby(['year-month'])[['past_thirtythree_month_return']].apply(modified_z)
df['past_thirtyfour_month_return_zscore'] = df.groupby(['year-month'])[['past_thirtyfour_month_return']].apply(modified_z)
df['past_thirtyfive_month_return_zscore'] = df.groupby(['year-month'])[['past_thirtyfive_month_return']].apply(modified_z)
df['past_thirtysix_month_return_zscore'] = df.groupby(['year-month'])[['past_thirtysix_month_return']].apply(modified_z)
df['accrual_zscore'] = df.groupby(['year-month'])[['accrual']].apply(modified_z)
df['adv_sale_zscore'] = df.groupby(['year-month'])[['adv_sale']].apply(modified_z)
df['aftret_eq_zscore'] = df.groupby(['year-month'])[['aftret_eq']].apply(modified_z)
df['aftret_equity_zscore'] = df.groupby(['year-month'])[['aftret_equity']].apply(modified_z)
df['aftret_invcapx_zscore'] = df.groupby(['year-month'])[['aftret_invcapx']].apply(modified_z)
df['at_turn_zscore'] = df.groupby(['year-month'])[['at_turn']].apply(modified_z)
df['bm_zscore'] = df.groupby(['year-month'])[['bm']].apply(modified_z)
df['CAPEI_zscore'] = df.groupby(['year-month'])[['CAPEI']].apply(modified_z)
df['capital_ratio_zscore'] = df.groupby(['year-month'])[['capital_ratio']].apply(modified_z)
df['cash_conversion_zscore'] = df.groupby(['year-month'])[['cash_conversion']].apply(modified_z)
df['cash_debt_zscore'] = df.groupby(['year-month'])[['cash_debt']].apply(modified_z)
df['cash_lt_zscore'] = df.groupby(['year-month'])[['cash_lt']].apply(modified_z)
df['cash_ratio_zscore'] = df.groupby(['year-month'])[['cash_ratio']].apply(modified_z)
df['cfm_zscore'] = df.groupby(['year-month'])[['cfm']].apply(modified_z)
df['curr_debt_zscore'] = df.groupby(['year-month'])[['curr_debt']].apply(modified_z)
df['curr_ratio_zscore'] = df.groupby(['year-month'])[['curr_ratio']].apply(modified_z)
df['de_ratio_zscore'] = df.groupby(['year-month'])[['de_ratio']].apply(modified_z)
df['debt_assets_zscore'] = df.groupby(['year-month'])[['debt_assets']].apply(modified_z)
df['debt_at_zscore'] = df.groupby(['year-month'])[['debt_at']].apply(modified_z)
df['debt_capital_zscore'] = df.groupby(['year-month'])[['debt_capital']].apply(modified_z)
df['debt_ebitda_zscore'] = df.groupby(['year-month'])[['debt_ebitda']].apply(modified_z)
df['debt_invcap_zscore'] = df.groupby(['year-month'])[['debt_invcap']].apply(modified_z)
df['dltt_be_zscore'] = df.groupby(['year-month'])[['dltt_be']].apply(modified_z)
df['dpr_zscore'] = df.groupby(['year-month'])[['dpr']].apply(modified_z)
df['efftax_zscore'] = df.groupby(['year-month'])[['efftax']].apply(modified_z)
df['equity_invcap_zscore'] = df.groupby(['year-month'])[['equity_invcap']].apply(modified_z)
df['evm_zscore'] = df.groupby(['year-month'])[['evm']].apply(modified_z)
df['fcf_ocf_zscore'] = df.groupby(['year-month'])[['fcf_ocf']].apply(modified_z)
df['gpm_zscore'] = df.groupby(['year-month'])[['gpm']].apply(modified_z)
df['GProf_zscore'] = df.groupby(['year-month'])[['GProf']].apply(modified_z)
df['int_debt_zscore'] = df.groupby(['year-month'])[['int_debt']].apply(modified_z)
df['int_totdebt_zscore'] = df.groupby(['year-month'])[['int_totdebt']].apply(modified_z)
df['intcov_zscore'] = df.groupby(['year-month'])[['intcov']].apply(modified_z)
df['intcov_ratio_zscore'] = df.groupby(['year-month'])[['intcov_ratio']].apply(modified_z)
df['inv_turn_zscore'] = df.groupby(['year-month'])[['inv_turn']].apply(modified_z)
df['invt_act_zscore'] = df.groupby(['year-month'])[['invt_act']].apply(modified_z)
df['lt_debt_zscore'] = df.groupby(['year-month'])[['lt_debt']].apply(modified_z)
df['lt_ppent_zscore'] = df.groupby(['year-month'])[['lt_ppent']].apply(modified_z)
df['npm_zscore'] = df.groupby(['year-month'])[['npm']].apply(modified_z)
df['ocf_lct_zscore'] = df.groupby(['year-month'])[['ocf_lct']].apply(modified_z)
df['opmad_zscore'] = df.groupby(['year-month'])[['opmad']].apply(modified_z)
df['opmbd_zscore'] = df.groupby(['year-month'])[['opmbd']].apply(modified_z)
df['pay_turn_zscore'] = df.groupby(['year-month'])[['pay_turn']].apply(modified_z)
df['pcf_zscore'] = df.groupby(['year-month'])[['pcf']].apply(modified_z)
df['pe_exi_zscore'] = df.groupby(['year-month'])[['pe_exi']].apply(modified_z)
df['pe_inc_zscore'] = df.groupby(['year-month'])[['pe_inc']].apply(modified_z)
df['pe_op_basic_zscore'] = df.groupby(['year-month'])[['pe_op_basic']].apply(modified_z)
df['pe_op_dil_zscore'] = df.groupby(['year-month'])[['pe_op_dil']].apply(modified_z)
df['PEG_1yrforward_zscore'] = df.groupby(['year-month'])[['PEG_1yrforward']].apply(modified_z)
df['PEG_ltgforward_zscore'] = df.groupby(['year-month'])[['PEG_ltgforward']].apply(modified_z)
df['PEG_trailing_zscore'] = df.groupby(['year-month'])[['PEG_trailing']].apply(modified_z)
df['pretret_earnat_zscore'] = df.groupby(['year-month'])[['pretret_earnat']].apply(modified_z)
df['pretret_noa_zscore'] = df.groupby(['year-month'])[['pretret_noa']].apply(modified_z)
df['profit_lct_zscore'] = df.groupby(['year-month'])[['profit_lct']].apply(modified_z)
df['ps_zscore'] = df.groupby(['year-month'])[['ps']].apply(modified_z)
df['ptb_zscore'] = df.groupby(['year-month'])[['ptb']].apply(modified_z)
df['ptpm_zscore'] = df.groupby(['year-month'])[['ptpm']].apply(modified_z)
df['quick_ratio_zscore'] = df.groupby(['year-month'])[['quick_ratio']].apply(modified_z)
df['rd_sale_zscore'] = df.groupby(['year-month'])[['rd_sale']].apply(modified_z)
df['rect_act_zscore'] = df.groupby(['year-month'])[['rect_act']].apply(modified_z)
df['rect_turn_zscore'] = df.groupby(['year-month'])[['rect_turn']].apply(modified_z)
df['roa_zscore'] = df.groupby(['year-month'])[['roa']].apply(modified_z)
df['roce_zscore'] = df.groupby(['year-month'])[['roce']].apply(modified_z)
df['roe_zscore'] = df.groupby(['year-month'])[['roe']].apply(modified_z)
df['sale_equity_zscore'] = df.groupby(['year-month'])[['sale_equity']].apply(modified_z)
df['sale_invcap_zscore'] = df.groupby(['year-month'])[['sale_invcap']].apply(modified_z)
df['sale_nwc_zscore'] = df.groupby(['year-month'])[['sale_nwc']].apply(modified_z)
df['short_debt_zscore'] = df.groupby(['year-month'])[['short_debt']].apply(modified_z)
df['staff_sale_zscore'] = df.groupby(['year-month'])[['staff_sale']].apply(modified_z)
df['totdebt_invcap_zscore'] = df.groupby(['year-month'])[['totdebt_invcap']].apply(modified_z)
df['alpha_zscore'] = df.groupby(['year-month'])[['alpha']].apply(modified_z)
df['b_hml_zscore'] = df.groupby(['year-month'])[['b_hml']].apply(modified_z)
df['b_mkt_zscore'] = df.groupby(['year-month'])[['b_mkt']].apply(modified_z)
df['b_smb_zscore'] = df.groupby(['year-month'])[['b_smb']].apply(modified_z)
df['b_umd_zscore'] = df.groupby(['year-month'])[['b_umd']].apply(modified_z)
df['exret_zscore'] = df.groupby(['year-month'])[['exret']].apply(modified_z)
df['ivol_zscore'] = df.groupby(['year-month'])[['ivol']].apply(modified_z)
df['n_zscore'] = df.groupby(['year-month'])[['n']].apply(modified_z)
df['R2_zscore'] = df.groupby(['year-month'])[['R2']].apply(modified_z)
df['tvol_zscore'] = df.groupby(['year-month'])[['tvol']].apply(modified_z)
df['BUYPCT_zscore'] = df.groupby(['year-month'])[['BUYPCT']].apply(modified_z)
df['HOLDPCT_zscore'] = df.groupby(['year-month'])[['HOLDPCT']].apply(modified_z)
df['MEANREC_zscore'] = df.groupby(['year-month'])[['MEANREC']].apply(modified_z)
df['MEDREC_zscore'] = df.groupby(['year-month'])[['MEDREC']].apply(modified_z)
df['recup_zscore'] = df.groupby(['year-month'])[['recup']].apply(modified_z)
df['recdown_zscore'] = df.groupby(['year-month'])[['recdown']].apply(modified_z)
df['SELLPCT_zscore'] = df.groupby(['year-month'])[['SELLPCT']].apply(modified_z)
df['STDEV_zscore'] = df.groupby(['year-month'])[['STDEV']].apply(modified_z)

C:\Users\sconner\AppData\Local\Continuum\Anaconda3\lib\site-packages\numpy\lib\nanfunctions.py:717: RuntimeWarning: All-NaN slice encountered
  warnings.warn("All-NaN slice encountered", RuntimeWarning)


In [56]:
# df - Enrich - Sector Z-Score
df['past_one_month_return_sector_zscore'] = df.groupby(['year-month', 'sector'])[['past_one_month_return']].apply(modified_z)
df['past_two_month_return_sector_zscore'] = df.groupby(['year-month', 'sector'])[['past_two_month_return']].apply(modified_z)
df['past_three_month_return_sector_zscore'] = df.groupby(['year-month', 'sector'])[['past_three_month_return']].apply(modified_z)
df['past_four_month_return_sector_zscore'] = df.groupby(['year-month', 'sector'])[['past_four_month_return']].apply(modified_z)
df['past_five_month_return_sector_zscore'] = df.groupby(['year-month', 'sector'])[['past_five_month_return']].apply(modified_z)
df['past_six_month_return_sector_zscore'] = df.groupby(['year-month', 'sector'])[['past_six_month_return']].apply(modified_z)
df['past_seven_month_return_sector_zscore'] = df.groupby(['year-month', 'sector'])[['past_seven_month_return']].apply(modified_z)
df['past_eight_month_return_sector_zscore'] = df.groupby(['year-month', 'sector'])[['past_eight_month_return']].apply(modified_z)
df['past_nine_month_return_sector_zscore'] = df.groupby(['year-month', 'sector'])[['past_nine_month_return']].apply(modified_z)
df['past_ten_month_return_sector_zscore'] = df.groupby(['year-month', 'sector'])[['past_ten_month_return']].apply(modified_z)
df['past_eleven_month_return_sector_zscore'] = df.groupby(['year-month', 'sector'])[['past_eleven_month_return']].apply(modified_z)
df['past_twelve_month_return_sector_zscore'] = df.groupby(['year-month', 'sector'])[['past_twelve_month_return']].apply(modified_z)
df['past_thirteen_month_return_sector_zscore'] = df.groupby(['year-month', 'sector'])[['past_thirteen_month_return']].apply(modified_z)
df['past_fourteen_month_return_sector_zscore'] = df.groupby(['year-month', 'sector'])[['past_fourteen_month_return']].apply(modified_z)
df['past_fifteen_month_return_sector_zscore'] = df.groupby(['year-month', 'sector'])[['past_fifteen_month_return']].apply(modified_z)
df['past_sixteen_month_return_sector_zscore'] = df.groupby(['year-month', 'sector'])[['past_sixteen_month_return']].apply(modified_z)
df['past_seventeen_month_return_sector_zscore'] = df.groupby(['year-month', 'sector'])[['past_seventeen_month_return']].apply(modified_z)
df['past_eighteen_month_return_sector_zscore'] = df.groupby(['year-month', 'sector'])[['past_eighteen_month_return']].apply(modified_z)
df['past_nineteen_month_return_sector_zscore'] = df.groupby(['year-month', 'sector'])[['past_nineteen_month_return']].apply(modified_z)
df['past_twenty_month_return_sector_zscore'] = df.groupby(['year-month', 'sector'])[['past_twenty_month_return']].apply(modified_z)
df['past_twentyone_month_return_sector_zscore'] = df.groupby(['year-month', 'sector'])[['past_twentyone_month_return']].apply(modified_z)
df['past_twentytwo_month_return_sector_zscore'] = df.groupby(['year-month', 'sector'])[['past_twentytwo_month_return']].apply(modified_z)
df['past_twentythree_month_return_sector_zscore'] = df.groupby(['year-month', 'sector'])[['past_twentythree_month_return']].apply(modified_z)
df['past_twentyfour_month_return_sector_zscore'] = df.groupby(['year-month', 'sector'])[['past_twentyfour_month_return']].apply(modified_z)
df['past_twentyfive_month_return_sector_zscore'] = df.groupby(['year-month', 'sector'])[['past_twentyfive_month_return']].apply(modified_z)
df['past_twentysix_month_return_sector_zscore'] = df.groupby(['year-month', 'sector'])[['past_twentysix_month_return']].apply(modified_z)
df['past_twentyseven_month_return_sector_zscore'] = df.groupby(['year-month', 'sector'])[['past_twentyseven_month_return']].apply(modified_z)
df['past_twentyeight_month_return_sector_zscore'] = df.groupby(['year-month', 'sector'])[['past_twentyeight_month_return']].apply(modified_z)
df['past_twentynine_month_return_sector_zscore'] = df.groupby(['year-month', 'sector'])[['past_twentynine_month_return']].apply(modified_z)
df['past_thirty_month_return_sector_zscore'] = df.groupby(['year-month', 'sector'])[['past_thirty_month_return']].apply(modified_z)
df['past_thirtyone_month_return_sector_zscore'] = df.groupby(['year-month', 'sector'])[['past_thirtyone_month_return']].apply(modified_z)
df['past_thirtytwo_month_return_sector_zscore'] = df.groupby(['year-month', 'sector'])[['past_thirtytwo_month_return']].apply(modified_z)
df['past_thirtythree_month_return_sector_zscore'] = df.groupby(['year-month', 'sector'])[['past_thirtythree_month_return']].apply(modified_z)
df['past_thirtyfour_month_return_sector_zscore'] = df.groupby(['year-month', 'sector'])[['past_thirtyfour_month_return']].apply(modified_z)
df['past_thirtyfive_month_return_sector_zscore'] = df.groupby(['year-month', 'sector'])[['past_thirtyfive_month_return']].apply(modified_z)
df['past_thirtysix_month_return_sector_zscore'] = df.groupby(['year-month', 'sector'])[['past_thirtysix_month_return']].apply(modified_z)
df['accrual_sector_zscore'] = df.groupby(['year-month', 'sector'])[['accrual']].apply(modified_z)
df['adv_sale_sector_zscore'] = df.groupby(['year-month', 'sector'])[['adv_sale']].apply(modified_z)
df['aftret_eq_sector_zscore'] = df.groupby(['year-month', 'sector'])[['aftret_eq']].apply(modified_z)
df['aftret_equity_sector_zscore'] = df.groupby(['year-month', 'sector'])[['aftret_equity']].apply(modified_z)
df['aftret_invcapx_sector_zscore'] = df.groupby(['year-month', 'sector'])[['aftret_invcapx']].apply(modified_z)
df['at_turn_sector_zscore'] = df.groupby(['year-month', 'sector'])[['at_turn']].apply(modified_z)
df['bm_sector_zscore'] = df.groupby(['year-month', 'sector'])[['bm']].apply(modified_z)
df['CAPEI_sector_zscore'] = df.groupby(['year-month', 'sector'])[['CAPEI']].apply(modified_z)
df['capital_ratio_sector_zscore'] = df.groupby(['year-month', 'sector'])[['capital_ratio']].apply(modified_z)
df['cash_conversion_sector_zscore'] = df.groupby(['year-month', 'sector'])[['cash_conversion']].apply(modified_z)
df['cash_debt_sector_zscore'] = df.groupby(['year-month', 'sector'])[['cash_debt']].apply(modified_z)
df['cash_lt_sector_zscore'] = df.groupby(['year-month', 'sector'])[['cash_lt']].apply(modified_z)
df['cash_ratio_sector_zscore'] = df.groupby(['year-month', 'sector'])[['cash_ratio']].apply(modified_z)
df['cfm_sector_zscore'] = df.groupby(['year-month', 'sector'])[['cfm']].apply(modified_z)
df['curr_debt_sector_zscore'] = df.groupby(['year-month', 'sector'])[['curr_debt']].apply(modified_z)
df['curr_ratio_sector_zscore'] = df.groupby(['year-month', 'sector'])[['curr_ratio']].apply(modified_z)
df['de_ratio_sector_zscore'] = df.groupby(['year-month', 'sector'])[['de_ratio']].apply(modified_z)
df['debt_assets_sector_zscore'] = df.groupby(['year-month', 'sector'])[['debt_assets']].apply(modified_z)
df['debt_at_sector_zscore'] = df.groupby(['year-month', 'sector'])[['debt_at']].apply(modified_z)
df['debt_capital_sector_zscore'] = df.groupby(['year-month', 'sector'])[['debt_capital']].apply(modified_z)
df['debt_ebitda_sector_zscore'] = df.groupby(['year-month', 'sector'])[['debt_ebitda']].apply(modified_z)
df['debt_invcap_sector_zscore'] = df.groupby(['year-month', 'sector'])[['debt_invcap']].apply(modified_z)
df['dltt_be_sector_zscore'] = df.groupby(['year-month', 'sector'])[['dltt_be']].apply(modified_z)
df['dpr_sector_zscore'] = df.groupby(['year-month', 'sector'])[['dpr']].apply(modified_z)
df['efftax_sector_zscore'] = df.groupby(['year-month', 'sector'])[['efftax']].apply(modified_z)
df['equity_invcap_sector_zscore'] = df.groupby(['year-month', 'sector'])[['equity_invcap']].apply(modified_z)
df['evm_sector_zscore'] = df.groupby(['year-month', 'sector'])[['evm']].apply(modified_z)
df['fcf_ocf_sector_zscore'] = df.groupby(['year-month', 'sector'])[['fcf_ocf']].apply(modified_z)
df['gpm_sector_zscore'] = df.groupby(['year-month', 'sector'])[['gpm']].apply(modified_z)
df['GProf_sector_zscore'] = df.groupby(['year-month', 'sector'])[['GProf']].apply(modified_z)
df['int_debt_sector_zscore'] = df.groupby(['year-month', 'sector'])[['int_debt']].apply(modified_z)
df['int_totdebt_sector_zscore'] = df.groupby(['year-month', 'sector'])[['int_totdebt']].apply(modified_z)
df['intcov_sector_zscore'] = df.groupby(['year-month', 'sector'])[['intcov']].apply(modified_z)
df['intcov_ratio_sector_zscore'] = df.groupby(['year-month', 'sector'])[['intcov_ratio']].apply(modified_z)
df['inv_turn_sector_zscore'] = df.groupby(['year-month', 'sector'])[['inv_turn']].apply(modified_z)
df['invt_act_sector_zscore'] = df.groupby(['year-month', 'sector'])[['invt_act']].apply(modified_z)
df['lt_debt_sector_zscore'] = df.groupby(['year-month', 'sector'])[['lt_debt']].apply(modified_z)
df['lt_ppent_sector_zscore'] = df.groupby(['year-month', 'sector'])[['lt_ppent']].apply(modified_z)
df['npm_sector_zscore'] = df.groupby(['year-month', 'sector'])[['npm']].apply(modified_z)
df['ocf_lct_sector_zscore'] = df.groupby(['year-month', 'sector'])[['ocf_lct']].apply(modified_z)
df['opmad_sector_zscore'] = df.groupby(['year-month', 'sector'])[['opmad']].apply(modified_z)
df['opmbd_sector_zscore'] = df.groupby(['year-month', 'sector'])[['opmbd']].apply(modified_z)
df['pay_turn_sector_zscore'] = df.groupby(['year-month', 'sector'])[['pay_turn']].apply(modified_z)
df['pcf_sector_zscore'] = df.groupby(['year-month', 'sector'])[['pcf']].apply(modified_z)
df['pe_exi_sector_zscore'] = df.groupby(['year-month', 'sector'])[['pe_exi']].apply(modified_z)
df['pe_inc_sector_zscore'] = df.groupby(['year-month', 'sector'])[['pe_inc']].apply(modified_z)
df['pe_op_basic_sector_zscore'] = df.groupby(['year-month', 'sector'])[['pe_op_basic']].apply(modified_z)
df['pe_op_dil_sector_zscore'] = df.groupby(['year-month', 'sector'])[['pe_op_dil']].apply(modified_z)
df['PEG_1yrforward_sector_zscore'] = df.groupby(['year-month', 'sector'])[['PEG_1yrforward']].apply(modified_z)
df['PEG_ltgforward_sector_zscore'] = df.groupby(['year-month', 'sector'])[['PEG_ltgforward']].apply(modified_z)
df['PEG_trailing_sector_zscore'] = df.groupby(['year-month', 'sector'])[['PEG_trailing']].apply(modified_z)
df['pretret_earnat_sector_zscore'] = df.groupby(['year-month', 'sector'])[['pretret_earnat']].apply(modified_z)
df['pretret_noa_sector_zscore'] = df.groupby(['year-month', 'sector'])[['pretret_noa']].apply(modified_z)
df['profit_lct_sector_zscore'] = df.groupby(['year-month', 'sector'])[['profit_lct']].apply(modified_z)
df['ps_sector_zscore'] = df.groupby(['year-month', 'sector'])[['ps']].apply(modified_z)
df['ptb_sector_zscore'] = df.groupby(['year-month', 'sector'])[['ptb']].apply(modified_z)
df['ptpm_sector_zscore'] = df.groupby(['year-month', 'sector'])[['ptpm']].apply(modified_z)
df['quick_ratio_sector_zscore'] = df.groupby(['year-month', 'sector'])[['quick_ratio']].apply(modified_z)
df['rd_sale_sector_zscore'] = df.groupby(['year-month', 'sector'])[['rd_sale']].apply(modified_z)
df['rect_act_sector_zscore'] = df.groupby(['year-month', 'sector'])[['rect_act']].apply(modified_z)
df['rect_turn_sector_zscore'] = df.groupby(['year-month', 'sector'])[['rect_turn']].apply(modified_z)
df['roa_sector_zscore'] = df.groupby(['year-month', 'sector'])[['roa']].apply(modified_z)
df['roce_sector_zscore'] = df.groupby(['year-month', 'sector'])[['roce']].apply(modified_z)
df['roe_sector_zscore'] = df.groupby(['year-month', 'sector'])[['roe']].apply(modified_z)
df['sale_equity_sector_zscore'] = df.groupby(['year-month', 'sector'])[['sale_equity']].apply(modified_z)
df['sale_invcap_sector_zscore'] = df.groupby(['year-month', 'sector'])[['sale_invcap']].apply(modified_z)
df['sale_nwc_sector_zscore'] = df.groupby(['year-month', 'sector'])[['sale_nwc']].apply(modified_z)
df['short_debt_sector_zscore'] = df.groupby(['year-month', 'sector'])[['short_debt']].apply(modified_z)
df['staff_sale_sector_zscore'] = df.groupby(['year-month', 'sector'])[['staff_sale']].apply(modified_z)
df['totdebt_invcap_sector_zscore'] = df.groupby(['year-month', 'sector'])[['totdebt_invcap']].apply(modified_z)
df['alpha_sector_zscore'] = df.groupby(['year-month', 'sector'])[['alpha']].apply(modified_z)
df['b_hml_sector_zscore'] = df.groupby(['year-month', 'sector'])[['b_hml']].apply(modified_z)
df['b_mkt_sector_zscore'] = df.groupby(['year-month', 'sector'])[['b_mkt']].apply(modified_z)
df['b_smb_sector_zscore'] = df.groupby(['year-month', 'sector'])[['b_smb']].apply(modified_z)
df['b_umd_sector_zscore'] = df.groupby(['year-month', 'sector'])[['b_umd']].apply(modified_z)
df['exret_sector_zscore'] = df.groupby(['year-month', 'sector'])[['exret']].apply(modified_z)
df['ivol_sector_zscore'] = df.groupby(['year-month', 'sector'])[['ivol']].apply(modified_z)
df['n_sector_zscore'] = df.groupby(['year-month', 'sector'])[['n']].apply(modified_z)
df['R2_sector_zscore'] = df.groupby(['year-month', 'sector'])[['R2']].apply(modified_z)
df['tvol_sector_zscore'] = df.groupby(['year-month', 'sector'])[['tvol']].apply(modified_z)
df['BUYPCT_sector_zscore'] = df.groupby(['year-month', 'sector'])[['BUYPCT']].apply(modified_z)
df['HOLDPCT_sector_zscore'] = df.groupby(['year-month', 'sector'])[['HOLDPCT']].apply(modified_z)
df['MEANREC_sector_zscore'] = df.groupby(['year-month', 'sector'])[['MEANREC']].apply(modified_z)
df['MEDREC_sector_zscore'] = df.groupby(['year-month', 'sector'])[['MEDREC']].apply(modified_z)
df['recup_sector_zscore'] = df.groupby(['year-month', 'sector'])[['recup']].apply(modified_z)
df['recdown_sector_zscore'] = df.groupby(['year-month', 'sector'])[['recdown']].apply(modified_z)
df['SELLPCT_sector_zscore'] = df.groupby(['year-month', 'sector'])[['SELLPCT']].apply(modified_z)
df['STDEV_sector_zscore'] = df.groupby(['year-month', 'sector'])[['STDEV']].apply(modified_z)

C:\Users\sconner\AppData\Local\Continuum\Anaconda3\lib\site-packages\numpy\lib\nanfunctions.py:717: RuntimeWarning: All-NaN slice encountered
  warnings.warn("All-NaN slice encountered", RuntimeWarning)


In [57]:
# df - Subset - Relevant features with adequate data

df = df[[
        'GVKEY', 
        'sector', 
        'year-month', 
        'forward_one_month_return', 
        'forward_two_month_return', 
        'forward_three_month_return', 
        'forward_four_month_return', 
        'forward_five_month_return', 
        'forward_six_month_return', 
        'forward_seven_month_return', 
        'forward_eight_month_return', 
        'forward_nine_month_return', 
        'forward_ten_month_return', 
        'forward_eleven_month_return', 
        'forward_twelve_month_return', 
        'forward_thirteen_month_return', 
        'forward_fourteen_month_return', 
        'forward_fifteen_month_return', 
        'forward_sixteen_month_return', 
        'forward_seventeen_month_return', 
        'forward_eighteen_month_return', 
        'forward_nineteen_month_return', 
        'forward_twenty_month_return', 
        'forward_twentyone_month_return', 
        'forward_twentytwo_month_return', 
        'forward_twentythree_month_return', 
        'forward_twentyfour_month_return', 
        'forward_twentyfive_month_return', 
        'forward_twentysix_month_return', 
        'forward_twentyseven_month_return', 
        'forward_twentyeight_month_return', 
        'forward_twentynine_month_return', 
        'forward_thirty_month_return', 
        'forward_thirtyone_month_return', 
        'forward_thirtytwo_month_return', 
        'forward_thirtythree_month_return', 
        'forward_thirtyfour_month_return', 
        'forward_thirtyfive_month_return', 
        'forward_thirtysix_month_return', 
        'january', 
        'february', 
        'march', 
        'april', 
        'may', 
        'june', 
        'july', 
        'august', 
        'september', 
        'october', 
        'november', 
        'december', 
        'past_one_month_return', 
        'past_two_month_return', 
        'past_three_month_return', 
        'past_four_month_return', 
        'past_five_month_return', 
        'past_six_month_return', 
        'past_seven_month_return', 
        'past_eight_month_return', 
        'past_nine_month_return', 
        'past_ten_month_return', 
        'past_eleven_month_return', 
        'past_twelve_month_return', 
        'past_thirteen_month_return', 
        'past_fourteen_month_return', 
        'past_fifteen_month_return', 
        'past_sixteen_month_return', 
        'past_seventeen_month_return', 
        'past_eighteen_month_return', 
        'past_nineteen_month_return', 
        'past_twenty_month_return', 
        'past_twentyone_month_return', 
        'past_twentytwo_month_return', 
        'past_twentythree_month_return', 
        'past_twentyfour_month_return', 
        'past_twentyfive_month_return', 
        'past_twentysix_month_return', 
        'past_twentyseven_month_return', 
        'past_twentyeight_month_return', 
        'past_twentynine_month_return', 
        'past_thirty_month_return', 
        'past_thirtyone_month_return', 
        'past_thirtytwo_month_return', 
        'past_thirtythree_month_return', 
        'past_thirtyfour_month_return', 
        'past_thirtyfive_month_return', 
        'past_thirtysix_month_return', 
        'past_one_month_return_sector_zscore', 
        'past_one_month_return_zscore', 
        'past_two_month_return_sector_zscore', 
        'past_two_month_return_zscore', 
        'past_three_month_return_sector_zscore', 
        'past_three_month_return_zscore', 
        'past_four_month_return_sector_zscore', 
        'past_four_month_return_zscore', 
        'past_five_month_return_sector_zscore', 
        'past_five_month_return_zscore', 
        'past_six_month_return_sector_zscore', 
        'past_six_month_return_zscore', 
        'past_seven_month_return_sector_zscore', 
        'past_seven_month_return_zscore', 
        'past_eight_month_return_sector_zscore', 
        'past_eight_month_return_zscore', 
        'past_nine_month_return_sector_zscore', 
        'past_nine_month_return_zscore', 
        'past_ten_month_return_sector_zscore', 
        'past_ten_month_return_zscore', 
        'past_eleven_month_return_sector_zscore', 
        'past_eleven_month_return_zscore', 
        'past_twelve_month_return_sector_zscore', 
        'past_twelve_month_return_zscore', 
        'past_thirteen_month_return_sector_zscore', 
        'past_thirteen_month_return_zscore', 
        'past_fourteen_month_return_sector_zscore', 
        'past_fourteen_month_return_zscore', 
        'past_fifteen_month_return_sector_zscore', 
        'past_fifteen_month_return_zscore', 
        'past_sixteen_month_return_sector_zscore', 
        'past_sixteen_month_return_zscore', 
        'past_seventeen_month_return_sector_zscore', 
        'past_seventeen_month_return_zscore', 
        'past_eighteen_month_return_sector_zscore', 
        'past_eighteen_month_return_zscore', 
        'past_nineteen_month_return_sector_zscore', 
        'past_nineteen_month_return_zscore', 
        'past_twenty_month_return_sector_zscore', 
        'past_twenty_month_return_zscore', 
        'past_twentyone_month_return_sector_zscore', 
        'past_twentyone_month_return_zscore', 
        'past_twentytwo_month_return_sector_zscore', 
        'past_twentytwo_month_return_zscore', 
        'past_twentythree_month_return_sector_zscore', 
        'past_twentythree_month_return_zscore', 
        'past_twentyfour_month_return_sector_zscore', 
        'past_twentyfour_month_return_zscore', 
        'past_twentyfive_month_return_sector_zscore', 
        'past_twentyfive_month_return_zscore', 
        'past_twentysix_month_return_sector_zscore', 
        'past_twentysix_month_return_zscore', 
        'past_twentyseven_month_return_sector_zscore', 
        'past_twentyseven_month_return_zscore', 
        'past_twentyeight_month_return_sector_zscore', 
        'past_twentyeight_month_return_zscore', 
        'past_twentynine_month_return_sector_zscore', 
        'past_twentynine_month_return_zscore', 
        'past_thirty_month_return_sector_zscore', 
        'past_thirty_month_return_zscore', 
        'past_thirtyone_month_return_sector_zscore', 
        'past_thirtyone_month_return_zscore', 
        'past_thirtytwo_month_return_sector_zscore', 
        'past_thirtytwo_month_return_zscore', 
        'past_thirtythree_month_return_sector_zscore', 
        'past_thirtythree_month_return_zscore', 
        'past_thirtyfour_month_return_sector_zscore', 
        'past_thirtyfour_month_return_zscore', 
        'past_thirtyfive_month_return_sector_zscore', 
        'past_thirtyfive_month_return_zscore', 
        'past_thirtysix_month_return_sector_zscore', 
        'past_thirtysix_month_return_zscore', 
        'accrual_sector_zscore', 
        'accrual_zscore', 
        'adv_sale_sector_zscore', 
        'adv_sale_zscore', 
        'aftret_eq_sector_zscore', 
        'aftret_eq_zscore', 
        'aftret_equity_sector_zscore', 
        'aftret_equity_zscore', 
        'aftret_invcapx_sector_zscore', 
        'aftret_invcapx_zscore', 
        'alpha_sector_zscore', 
        'alpha_zscore', 
        'at_turn_sector_zscore', 
        'at_turn_zscore', 
        'b_hml_sector_zscore', 
        'b_hml_zscore', 
        'b_mkt_sector_zscore', 
        'b_mkt_zscore', 
        'b_smb_sector_zscore', 
        'b_smb_zscore', 
        'b_umd_sector_zscore', 
        'b_umd_zscore', 
        'bm_sector_zscore', 
        'bm_zscore', 
        'BUYPCT_sector_zscore', 
        'BUYPCT_zscore', 
        'CAPEI_sector_zscore', 
        'CAPEI_zscore', 
        'capital_ratio_sector_zscore', 
        'capital_ratio_zscore', 
        'cash_conversion_sector_zscore', 
        'cash_conversion_zscore', 
        'cash_debt_sector_zscore', 
        'cash_debt_zscore', 
        'cash_lt_sector_zscore', 
        'cash_lt_zscore', 
        'cash_ratio_sector_zscore', 
        'cash_ratio_zscore', 
        'cfm_sector_zscore', 
        'cfm_zscore', 
        'curr_debt_sector_zscore', 
        'curr_debt_zscore', 
        'curr_ratio_sector_zscore', 
        'curr_ratio_zscore', 
        'de_ratio_sector_zscore', 
        'de_ratio_zscore', 
        'debt_assets_sector_zscore', 
        'debt_assets_zscore', 
        'debt_at_sector_zscore', 
        'debt_at_zscore', 
        'debt_capital_sector_zscore', 
        'debt_capital_zscore', 
        'debt_ebitda_sector_zscore', 
        'debt_ebitda_zscore', 
        'debt_invcap_sector_zscore', 
        'debt_invcap_zscore', 
        'DIVYIELD', 
        'dltt_be_sector_zscore', 
        'dltt_be_zscore', 
        'dpr_sector_zscore', 
        'dpr_zscore', 
        'dvpspm', 
        'dvpsxm', 
        'dvrate', 
        'efftax_sector_zscore', 
        'efftax_zscore', 
        'equity_invcap_sector_zscore', 
        'equity_invcap_zscore', 
        'evm_sector_zscore', 
        'evm_zscore', 
        'exret_sector_zscore', 
        'exret_zscore', 
        'fcf_ocf_sector_zscore', 
        'fcf_ocf_zscore', 
        'gpm_sector_zscore', 
        'gpm_zscore', 
        'GProf_sector_zscore', 
        'GProf_zscore', 
        'HOLDPCT_sector_zscore', 
        'HOLDPCT_zscore', 
        'int_debt_sector_zscore', 
        'int_debt_zscore', 
        'int_totdebt_sector_zscore', 
        'int_totdebt_zscore', 
        'intcov_ratio_sector_zscore', 
        'intcov_ratio_zscore', 
        'intcov_sector_zscore', 
        'intcov_zscore', 
        'inv_turn_sector_zscore', 
        'inv_turn_zscore', 
        'invt_act_sector_zscore', 
        'invt_act_zscore', 
        'ivol_sector_zscore', 
        'ivol_zscore', 
        'lt_debt_sector_zscore', 
        'lt_debt_zscore', 
        'lt_ppent_sector_zscore', 
        'lt_ppent_zscore', 
        'MEANREC_sector_zscore', 
        'MEANREC_zscore', 
        'MEDREC_sector_zscore', 
        'MEDREC_zscore', 
        'n_sector_zscore', 
        'n_zscore', 
        'npm_sector_zscore', 
        'npm_zscore', 
        'ocf_lct_sector_zscore', 
        'ocf_lct_zscore', 
        'opmad_sector_zscore', 
        'opmad_zscore', 
        'opmbd_sector_zscore', 
        'opmbd_zscore', 
        'pay_turn_sector_zscore', 
        'pay_turn_zscore', 
        'pcf_sector_zscore', 
        'pcf_zscore', 
        'pe_exi_sector_zscore', 
        'pe_exi_zscore', 
        'pe_inc_sector_zscore', 
        'pe_inc_zscore', 
        'pe_op_basic_sector_zscore', 
        'pe_op_basic_zscore', 
        'pe_op_dil_sector_zscore', 
        'pe_op_dil_zscore', 
        'PEG_1yrforward_sector_zscore', 
        'PEG_1yrforward_zscore', 
        'PEG_ltgforward_sector_zscore', 
        'PEG_ltgforward_zscore', 
        'PEG_trailing_sector_zscore', 
        'PEG_trailing_zscore', 
        'pretret_earnat_sector_zscore', 
        'pretret_earnat_zscore', 
        'pretret_noa_sector_zscore', 
        'pretret_noa_zscore', 
        'profit_lct_sector_zscore', 
        'profit_lct_zscore', 
        'ps_sector_zscore', 
        'ps_zscore', 
        'ptb_sector_zscore', 
        'ptb_zscore', 
        'ptpm_sector_zscore', 
        'ptpm_zscore', 
        'quick_ratio_sector_zscore', 
        'quick_ratio_zscore', 
        'R2_sector_zscore', 
        'R2_zscore', 
        'rd_sale_sector_zscore', 
        'rd_sale_zscore', 
        'recdown_sector_zscore', 
        'recdown_zscore', 
        'rect_act_sector_zscore', 
        'rect_act_zscore', 
        'rect_turn_sector_zscore', 
        'rect_turn_zscore', 
        'recup_sector_zscore', 
        'recup_zscore', 
        'roa_sector_zscore', 
        'roa_zscore', 
        'roce_sector_zscore', 
        'roce_zscore', 
        'roe_sector_zscore', 
        'roe_zscore', 
        'sale_equity_sector_zscore', 
        'sale_equity_zscore', 
        'sale_invcap_sector_zscore', 
        'sale_invcap_zscore', 
        'sale_nwc_sector_zscore', 
        'sale_nwc_zscore', 
        'SELLPCT_sector_zscore', 
        'SELLPCT_zscore', 
        'short_debt_sector_zscore', 
        'short_debt_zscore', 
        'spcsrc',
        'staff_sale_sector_zscore', 
        'staff_sale_zscore', 
        'STDEV_sector_zscore', 
        'STDEV_zscore', 
        'totdebt_invcap_sector_zscore', 
        'totdebt_invcap_zscore', 
        'tvol_sector_zscore', 
        'tvol_zscore'
        ]]

In [58]:
y = df[[
            'forward_one_month_return', 
            'forward_two_month_return', 
            'forward_three_month_return', 
            'forward_four_month_return', 
            'forward_five_month_return', 
            'forward_six_month_return', 
            'forward_seven_month_return', 
            'forward_eight_month_return', 
            'forward_nine_month_return', 
            'forward_ten_month_return', 
            'forward_eleven_month_return', 
            'forward_twelve_month_return', 
            'forward_thirteen_month_return', 
            'forward_fourteen_month_return', 
            'forward_fifteen_month_return', 
            'forward_sixteen_month_return', 
            'forward_seventeen_month_return', 
            'forward_eighteen_month_return', 
            'forward_nineteen_month_return', 
            'forward_twenty_month_return', 
            'forward_twentyone_month_return', 
            'forward_twentytwo_month_return', 
            'forward_twentythree_month_return', 
            'forward_twentyfour_month_return', 
            'forward_twentyfive_month_return', 
            'forward_twentysix_month_return', 
            'forward_twentyseven_month_return', 
            'forward_twentyeight_month_return', 
            'forward_twentynine_month_return', 
            'forward_thirty_month_return', 
            'forward_thirtyone_month_return', 
            'forward_thirtytwo_month_return', 
            'forward_thirtythree_month_return', 
            'forward_thirtyfour_month_return', 
            'forward_thirtyfive_month_return', 
            'forward_thirtysix_month_return',
            ]]

In [59]:
months = df[[
        'january', 
        'february', 
        'march', 
        'april', 
        'may', 
        'june', 
        'july', 
        'august', 
        'september', 
        'october', 
        'november', 
        'december'
            ]]

In [60]:
metric = df[[
        'past_one_month_return', 
        'past_two_month_return', 
        'past_three_month_return', 
        'past_four_month_return', 
        'past_five_month_return', 
        'past_six_month_return', 
        'past_seven_month_return', 
        'past_eight_month_return', 
        'past_nine_month_return', 
        'past_ten_month_return', 
        'past_eleven_month_return', 
        'past_twelve_month_return', 
        'past_thirteen_month_return', 
        'past_fourteen_month_return', 
        'past_fifteen_month_return', 
        'past_sixteen_month_return', 
        'past_seventeen_month_return', 
        'past_eighteen_month_return', 
        'past_nineteen_month_return', 
        'past_twenty_month_return', 
        'past_twentyone_month_return', 
        'past_twentytwo_month_return', 
        'past_twentythree_month_return', 
        'past_twentyfour_month_return', 
        'past_twentyfive_month_return', 
        'past_twentysix_month_return', 
        'past_twentyseven_month_return', 
        'past_twentyeight_month_return', 
        'past_twentynine_month_return', 
        'past_thirty_month_return', 
        'past_thirtyone_month_return', 
        'past_thirtytwo_month_return', 
        'past_thirtythree_month_return', 
        'past_thirtyfour_month_return', 
        'past_thirtyfive_month_return', 
        'past_thirtysix_month_return', 
        'past_one_month_return_sector_zscore', 
        'past_one_month_return_zscore', 
        'past_two_month_return_sector_zscore', 
        'past_two_month_return_zscore', 
        'past_three_month_return_sector_zscore', 
        'past_three_month_return_zscore', 
        'past_four_month_return_sector_zscore', 
        'past_four_month_return_zscore', 
        'past_five_month_return_sector_zscore', 
        'past_five_month_return_zscore', 
        'past_six_month_return_sector_zscore', 
        'past_six_month_return_zscore', 
        'past_seven_month_return_sector_zscore', 
        'past_seven_month_return_zscore', 
        'past_eight_month_return_sector_zscore', 
        'past_eight_month_return_zscore', 
        'past_nine_month_return_sector_zscore', 
        'past_nine_month_return_zscore', 
        'past_ten_month_return_sector_zscore', 
        'past_ten_month_return_zscore', 
        'past_eleven_month_return_sector_zscore', 
        'past_eleven_month_return_zscore', 
        'past_twelve_month_return_sector_zscore', 
        'past_twelve_month_return_zscore', 
        'past_thirteen_month_return_sector_zscore', 
        'past_thirteen_month_return_zscore', 
        'past_fourteen_month_return_sector_zscore', 
        'past_fourteen_month_return_zscore', 
        'past_fifteen_month_return_sector_zscore', 
        'past_fifteen_month_return_zscore', 
        'past_sixteen_month_return_sector_zscore', 
        'past_sixteen_month_return_zscore', 
        'past_seventeen_month_return_sector_zscore', 
        'past_seventeen_month_return_zscore', 
        'past_eighteen_month_return_sector_zscore', 
        'past_eighteen_month_return_zscore', 
        'past_nineteen_month_return_sector_zscore', 
        'past_nineteen_month_return_zscore', 
        'past_twenty_month_return_sector_zscore', 
        'past_twenty_month_return_zscore', 
        'past_twentyone_month_return_sector_zscore', 
        'past_twentyone_month_return_zscore', 
        'past_twentytwo_month_return_sector_zscore', 
        'past_twentytwo_month_return_zscore', 
        'past_twentythree_month_return_sector_zscore', 
        'past_twentythree_month_return_zscore', 
        'past_twentyfour_month_return_sector_zscore', 
        'past_twentyfour_month_return_zscore', 
        'past_twentyfive_month_return_sector_zscore', 
        'past_twentyfive_month_return_zscore', 
        'past_twentysix_month_return_sector_zscore', 
        'past_twentysix_month_return_zscore', 
        'past_twentyseven_month_return_sector_zscore', 
        'past_twentyseven_month_return_zscore', 
        'past_twentyeight_month_return_sector_zscore', 
        'past_twentyeight_month_return_zscore', 
        'past_twentynine_month_return_sector_zscore', 
        'past_twentynine_month_return_zscore', 
        'past_thirty_month_return_sector_zscore', 
        'past_thirty_month_return_zscore', 
        'past_thirtyone_month_return_sector_zscore', 
        'past_thirtyone_month_return_zscore', 
        'past_thirtytwo_month_return_sector_zscore', 
        'past_thirtytwo_month_return_zscore', 
        'past_thirtythree_month_return_sector_zscore', 
        'past_thirtythree_month_return_zscore', 
        'past_thirtyfour_month_return_sector_zscore', 
        'past_thirtyfour_month_return_zscore', 
        'past_thirtyfive_month_return_sector_zscore', 
        'past_thirtyfive_month_return_zscore', 
        'past_thirtysix_month_return_sector_zscore', 
        'past_thirtysix_month_return_zscore', 
        'accrual_sector_zscore', 
        'accrual_zscore', 
        'adv_sale_sector_zscore', 
        'adv_sale_zscore', 
        'aftret_eq_sector_zscore', 
        'aftret_eq_zscore', 
        'aftret_equity_sector_zscore', 
        'aftret_equity_zscore', 
        'aftret_invcapx_sector_zscore', 
        'aftret_invcapx_zscore', 
        'alpha_sector_zscore', 
        'alpha_zscore', 
        'at_turn_sector_zscore', 
        'at_turn_zscore', 
        'b_hml_sector_zscore', 
        'b_hml_zscore', 
        'b_mkt_sector_zscore', 
        'b_mkt_zscore', 
        'b_smb_sector_zscore', 
        'b_smb_zscore', 
        'b_umd_sector_zscore', 
        'b_umd_zscore', 
        'bm_sector_zscore', 
        'bm_zscore', 
        'BUYPCT_sector_zscore', 
        'BUYPCT_zscore', 
        'CAPEI_sector_zscore', 
        'CAPEI_zscore', 
        'capital_ratio_sector_zscore', 
        'capital_ratio_zscore', 
        'cash_conversion_sector_zscore', 
        'cash_conversion_zscore', 
        'cash_debt_sector_zscore', 
        'cash_debt_zscore', 
        'cash_lt_sector_zscore', 
        'cash_lt_zscore', 
        'cash_ratio_sector_zscore', 
        'cash_ratio_zscore', 
        'cfm_sector_zscore', 
        'cfm_zscore', 
        'curr_debt_sector_zscore', 
        'curr_debt_zscore', 
        'curr_ratio_sector_zscore', 
        'curr_ratio_zscore', 
        'de_ratio_sector_zscore', 
        'de_ratio_zscore', 
        'debt_assets_sector_zscore', 
        'debt_assets_zscore', 
        'debt_at_sector_zscore', 
        'debt_at_zscore', 
        'debt_capital_sector_zscore', 
        'debt_capital_zscore', 
        'debt_ebitda_sector_zscore', 
        'debt_ebitda_zscore', 
        'debt_invcap_sector_zscore', 
        'debt_invcap_zscore', 
        'DIVYIELD', 
        'dltt_be_sector_zscore', 
        'dltt_be_zscore', 
        'dpr_sector_zscore', 
        'dpr_zscore', 
        'dvpspm', 
        'dvpsxm', 
        'dvrate', 
        'efftax_sector_zscore', 
        'efftax_zscore', 
        'equity_invcap_sector_zscore', 
        'equity_invcap_zscore', 
        'evm_sector_zscore', 
        'evm_zscore', 
        'exret_sector_zscore', 
        'exret_zscore', 
        'fcf_ocf_sector_zscore', 
        'fcf_ocf_zscore', 
        'gpm_sector_zscore', 
        'gpm_zscore', 
        'GProf_sector_zscore', 
        'GProf_zscore', 
        'HOLDPCT_sector_zscore', 
        'HOLDPCT_zscore', 
        'int_debt_sector_zscore', 
        'int_debt_zscore', 
        'int_totdebt_sector_zscore', 
        'int_totdebt_zscore', 
        'intcov_ratio_sector_zscore', 
        'intcov_ratio_zscore', 
        'intcov_sector_zscore', 
        'intcov_zscore', 
        'inv_turn_sector_zscore', 
        'inv_turn_zscore', 
        'invt_act_sector_zscore', 
        'invt_act_zscore', 
        'ivol_sector_zscore', 
        'ivol_zscore', 
        'lt_debt_sector_zscore', 
        'lt_debt_zscore', 
        'lt_ppent_sector_zscore', 
        'lt_ppent_zscore', 
        'MEANREC_sector_zscore', 
        'MEANREC_zscore', 
        'MEDREC_sector_zscore', 
        'MEDREC_zscore', 
        'n_sector_zscore', 
        'n_zscore', 
        'npm_sector_zscore', 
        'npm_zscore', 
        'ocf_lct_sector_zscore', 
        'ocf_lct_zscore', 
        'opmad_sector_zscore', 
        'opmad_zscore', 
        'opmbd_sector_zscore', 
        'opmbd_zscore', 
        'pay_turn_sector_zscore', 
        'pay_turn_zscore', 
        'pcf_sector_zscore', 
        'pcf_zscore', 
        'pe_exi_sector_zscore', 
        'pe_exi_zscore', 
        'pe_inc_sector_zscore', 
        'pe_inc_zscore', 
        'pe_op_basic_sector_zscore', 
        'pe_op_basic_zscore', 
        'pe_op_dil_sector_zscore', 
        'pe_op_dil_zscore', 
        'PEG_1yrforward_sector_zscore', 
        'PEG_1yrforward_zscore', 
        'PEG_ltgforward_sector_zscore', 
        'PEG_ltgforward_zscore', 
        'PEG_trailing_sector_zscore', 
        'PEG_trailing_zscore', 
        'pretret_earnat_sector_zscore', 
        'pretret_earnat_zscore', 
        'pretret_noa_sector_zscore', 
        'pretret_noa_zscore', 
        'profit_lct_sector_zscore', 
        'profit_lct_zscore', 
        'ps_sector_zscore', 
        'ps_zscore', 
        'ptb_sector_zscore', 
        'ptb_zscore', 
        'ptpm_sector_zscore', 
        'ptpm_zscore', 
        'quick_ratio_sector_zscore', 
        'quick_ratio_zscore', 
        'R2_sector_zscore', 
        'R2_zscore', 
        'rd_sale_sector_zscore', 
        'rd_sale_zscore', 
        'recdown_sector_zscore', 
        'recdown_zscore', 
        'rect_act_sector_zscore', 
        'rect_act_zscore', 
        'rect_turn_sector_zscore', 
        'rect_turn_zscore', 
        'recup_sector_zscore', 
        'recup_zscore', 
        'roa_sector_zscore', 
        'roa_zscore', 
        'roce_sector_zscore', 
        'roce_zscore', 
        'roe_sector_zscore', 
        'roe_zscore', 
        'sale_equity_sector_zscore', 
        'sale_equity_zscore', 
        'sale_invcap_sector_zscore', 
        'sale_invcap_zscore', 
        'sale_nwc_sector_zscore', 
        'sale_nwc_zscore', 
        'SELLPCT_sector_zscore', 
        'SELLPCT_zscore', 
        'short_debt_sector_zscore', 
        'short_debt_zscore', 
        'spcsrc', 
        'staff_sale_sector_zscore', 
        'staff_sale_zscore', 
        'STDEV_sector_zscore', 
        'STDEV_zscore', 
        'totdebt_invcap_sector_zscore', 
        'totdebt_invcap_zscore', 
        'tvol_sector_zscore', 
        'tvol_zscore'
        ]]

In [61]:
# metric - Subset - Numeric Columns
metric_numerics = metric.select_dtypes(include=[np.float]).columns

In [62]:
metric = metric[metric_numerics].apply(clip_outliers)

In [63]:
metric = metric[metric_numerics].apply(fill_null)

C:\Users\sconner\AppData\Local\Continuum\Anaconda3\lib\site-packages\numpy\lib\nanfunctions.py:717: RuntimeWarning: All-NaN slice encountered
  warnings.warn("All-NaN slice encountered", RuntimeWarning)


In [64]:
X = pd.concat([metric, months], axis=1)

In [65]:
return_all_rows(df.describe().transpose())

C:\Users\sconner\AppData\Local\Continuum\Anaconda3\lib\site-packages\numpy\lib\function_base.py:3817: RuntimeWarning: invalid value encountered in multiply
  x2 = take(ap, indices_above, axis=axis) * weights_above


,count,mean,std,min,25%,50%,75%,max
GVKEY,47539.0,7.069949e+04,71125.349280,1.004000e+03,1.263500e+04,2.606900e+04,1.473490e+05,3.153180e+05
forward_one_month_return,43526.0,7.744155e+01,3376.106959,-9.993736e+01,-6.661865e+00,-3.221996e-01,5.559735e+00,1.510224e+05
forward_two_month_return,39515.0,1.567291e+01,1512.722422,-9.993663e+01,-9.843702e+00,-5.799503e-01,7.519293e+00,1.571258e+05
forward_three_month_return,35510.0,7.525333e+01,3353.783481,-9.993907e+01,-1.291303e+01,-9.524738e-01,8.830979e+00,1.567988e+05
forward_four_month_return,31535.0,1.621775e+01,1654.273516,-9.993622e+01,-1.620010e+01,-2.309859e+00,8.547325e+00,1.486995e+05
forward_five_month_return,27612.0,8.275792e+01,3560.845173,-9.993760e+01,-1.995651e+01,-3.758622e+00,8.975702e+00,1.543172e+05
forward_six_month_return,23724.0,1.933031e+01,1852.110290,-9.993742e+01,-2.192682e+01,-4.649209e+00,9.023031e+00,1.433428e+05
forward_seven_month_return,19881.0,9.952494e+01,3911.688183,-9.993929e+01,-2.481273e+01,-5.235602e+00,1.017223e+01,1.515871e+05
forward_eight_month_return,16072.0,3.080513e+01,2282.385848,-9.993846e+01,-2.515715e+01,-5.071970e+00,1.068990e+01,1.502167e+05
forward_nine_month_return,12289.0,1.387889e+02,4508.509912,-9.993889e+01,-2.609646e+01,-4.166667e+00,1.372391e+01,1.491600e+05


In [66]:
from scipy.stats.stats import pearsonr

In [67]:
feature = []
correlation = []
correlation_direction = []
significance = []

for column in df:
    pair = pearsonr(dependent['forward_one_month_return'], df[column])
    feature.append(column)
    correlation.append(abs(pair[0]))
    correlation_direction.append(pair[0])
    significance.append(pair[1])
    
review = pd.DataFrame()
review['feature'] = feature
review['correlation'] = correlation
review['correlation_direction'] = correlation_direction
review['significance'] = significance

# review - Sort - Correlation
review = review.sort_values(by=['correlation'], ascending=0)

NameError: name 'dependent' is not defined

In [ ]:
review

In [ ]:
# dependent_binary = dependent[dependent < 0] = 0 and dependent[dependent >= 0] = 1